In [1]:
import sys
from pathlib import Path

import pandas as pd
import yaml

sys.path.insert(0, "..")
from runner.utils import (
    allocate_benchmarks,
    allocate_vms_greedy,
    check_uptimes,
    create_benchmark_campaign,
    fetch_all_partial_results,
    load_benchmark_metadata,
    load_results,
)

In [ ]:
# If a util function was modified, use this cell to reload it without having to restart the kernel
%run ../runner/utils.py

### Utils for computing v1 runtimes (and disruptive name changes)

In [118]:
meta_path = Path("../results/metadata.yaml")
with open(meta_path, "r") as f:
    metadata = yaml.safe_load(f)

In [119]:
results_v1 = pd.read_csv("../results/benchmark_results.csv")

results_v1["bench-size"] = results_v1["Benchmark"] + "-" + results_v1["Size"]
results_v1["solver-version"] = results_v1["Solver"] + "-" + results_v1["Solver Version"]
results_v1.shape, len(results_v1["bench-size"].unique())

((1425, 19), 120)

In [120]:
# We will allocate the instances to many VMs using v1 runtimes:
total_v1_runtimes = results_v1.groupby("bench-size")["Runtime (s)"].sum()
total_v1_runtimes

bench-size
DCOPF-Carolinas_1W-1-997                    418.152434
DCOPF-Carolinas_2M-1-997                    419.125940
DCOPF-Carolinas_6M-1-997                    420.224717
DCOPF-Carolinas_uc_1W-1-997               34586.867308
DCOPF-Carolinas_uc_2M-1-997               32147.728408
                                              ...     
tulipa-1_EU_investment_simple-28-1h       54342.385765
tulipa-1_EU_investment_simple-28-2.2h     11003.532512
tulipa-1_EU_investment_simple-28-24h         43.360834
tulipa-1_EU_investment_simple-28-4.3h      9860.583211
tulipa-1_EU_investment_simple-28-52.1h      209.519901
Name: Runtime (s), Length: 120, dtype: float64

In [121]:
# Redo the disruptive name changes to obtain runtime from v1 results:
def disruptive_change_bench_size(name):
    if name.startswith("temoa"):
        return name + "ts"
    if name.startswith("Sienna") and name.endswith("-1-1h"):
        return name.replace("-1-1h", "-73-1h")
    if name.startswith("DCOPF-Carolinas") and name.endswith("-1-997"):
        return name.replace("-1-997", "-997-1h")
    changes = {
        "times-ireland-noco2-1-1h": "times-ireland-noco2-1-1ts",
        "times-ireland-noco2-40-1h": "times-ireland-noco2-40-1ts",
        "times-ireland-noco2-counties-26-1h": "times-ireland-noco2-counties-26-1ts",
        "times-nz-kea-2-24h": "times-nz-kea-2-24ts",
        "times-nz-tui-2-24h": "times-nz-tui-2-24ts",
    }
    return changes.get(name, name)


total_v1_runtimes.index = total_v1_runtimes.index.map(disruptive_change_bench_size)

In [122]:
# Create a benchmark instance DF from the metadata.yaml file TODO move to utils
rows = []
ignore_keys = {"Short description", "Realistic motivation", "Sizes", "Name"}
for bench_name, bench_data in metadata["benchmarks"].items():
    common_attrs = {k: v for k, v in bench_data.items() if k not in ignore_keys}
    for size_data in bench_data.get("Sizes", []):
        size_attrs = {k: v for k, v in size_data.items() if k not in ignore_keys}
        row = {
            "Benchmark": bench_name,
            "Instance": size_data.get("Name"),
            **common_attrs,
            **size_attrs,
        }
        rows.append(row)
benchmarks_df = pd.DataFrame(rows)
benchmarks_df.index = benchmarks_df["Benchmark"] + "-" + benchmarks_df["Instance"]
benchmarks_df["v1 runtime"] = total_v1_runtimes

# benchmarks_df[benchmarks_df['v1 runtime'].isna()][['Size', 'Num. variables', 'v1 runtime']]
benchmarks_df

,Benchmark,Instance,Modelling framework,Model name,Version,Contributor(s)/Source,License,Problem class,Application,Sectoral focus,...,Temporal resolution,Spatial resolution,Realistic,Num. constraints,Num. variables,Num. continuous variables,Num. integer variables,spatial resolution,Notes,v1 runtime
genx-elec_trex-15-168h,genx-elec_trex,15-168h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,168 hours,15 nodes,True,11832658.0,13777226.0,NaN,NaN,NaN,NaN,79441.437168
genx-elec_trex_co2-15-168h,genx-elec_trex_co2,15-168h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,168 hours,15 nodes,True,11832689.0,13777227.0,NaN,NaN,NaN,NaN,87509.210944
genx-elec_trex_uc-15-24h,genx-elec_trex_uc,15-24h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,MILP,Infrastructure & Capacity Expansion,Power-only,...,24h hours,15 nodes,True,3123898.0,2551346.0,2062938.0,488408.0,NaN,NaN,36850.939651
genx-elec_co2-15-168h,genx-elec_co2,15-168h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,168 hours,15 nodes,True,11832633.0,13777201.0,NaN,NaN,NaN,NaN,80570.337039
genx-1_three_zones-no_uc-3-1h,genx-1_three_zones-no_uc,3-1h,GenX,GenX,None,https://github.com/jump-dev/open-energy-modeli...,"Model code licensed MIT, benchmark instances n...",LP,Infrastructure & Capacity Expansion,Power-only,...,1 hour,3 nodes,False,114590.0,103506.0,NaN,NaN,NaN,NaN,4954.848739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
times-ireland-noco2-1-1ts,times-ireland-noco2,1-1ts,TIMES,TIMES-Ireland,None,Olexandr Balyk,CC BY 4.0,LP,Infrastructure & Capacity expansion,Sector-coupled,...,1 time slice,1 region,False,415261.0,544694.0,NaN,NaN,NaN,NaN,35457.170788
times-ireland-noco2-40-1ts,times-ireland-noco2,40-1ts,TIMES,TIMES-Ireland,None,Olexandr Balyk,CC BY 4.0,LP,Infrastructure & Capacity expansion,Sector-coupled,...,1 time slice,40 regions,False,4809283.0,4534121.0,NaN,NaN,NaN,NaN,74847.159706
times-ireland-noco2-counties-26-1ts,times-ireland-noco2-counties,26-1ts,TIMES,TIMES-Ireland,None,Olexandr Balyk,CC BY 4.0,LP,Infrastructure & Capacity expansion,Sector-coupled,...,1 time slice,26 regions (Transport) / 1 region (all other s...,False,873805.0,1235884.0,NaN,NaN,NaN,NaN,40600.785081
times-nz-kea-2-24ts,times-nz-kea,2-24ts,TIMES,TIMES-NZ,None,Olexandr Balyk,CC BY 4.0,LP,Infrastructure & Capacity expansion,Sector-coupled,...,24 time slices,2 regions,True,300078.0,202797.0,NaN,NaN,NaN,NaN,2666.392314


## Create benchmark campaign(s)

### 20251208 Test run on all S instances

In [ ]:
results = pd.read_csv("../results/benchmark_results.csv")

results["bench-size"] = results["Benchmark"] + "-" + results["Size"]
results["solver-version"] = results["Solver"] + "-" + results["Solver Version"]
results.shape, len(results["bench-size"].unique())

((1425, 19), 120)

In [ ]:
meta_path = Path("../results/metadata.yaml")
with open(meta_path, "r") as f:
    metadata = yaml.safe_load(f)

s_instances = {}
for bench_name, bench_data in metadata.get("benchmarks", {}).items():
    for size_data in bench_data.get("Sizes", []):
        if size_data["Size"] == "S":
            s_instances[bench_name + "-" + size_data["Name"]] = {
                "bench": bench_name,
                "inst": size_data["Name"],
                "class": bench_data["Problem class"],
                "size": size_data["Size"],
                "URL": size_data["URL"],
            }

# Throw out this one that no one could solve
del s_instances["pglib_opf_case1803_snem-1803-NA"]

len(s_instances)

17

In [ ]:
# Running all sequentially takes around 28h, so let's split into many VMs using v1 runtimes:
# naturaldelta(results[results['bench-size'].isin(s_instances)]['Runtime (s)'].sum())

total_v1_runtimes = results.groupby("bench-size")["Runtime (s)"].sum()
total_v1_runtimes

bench-size
DCOPF-Carolinas_1W-1-997                    418.152434
DCOPF-Carolinas_2M-1-997                    419.125940
DCOPF-Carolinas_6M-1-997                    420.224717
DCOPF-Carolinas_uc_1W-1-997               34586.867308
DCOPF-Carolinas_uc_2M-1-997               32147.728408
                                              ...     
tulipa-1_EU_investment_simple-28-1h       54342.385765
tulipa-1_EU_investment_simple-28-2.2h     11003.532512
tulipa-1_EU_investment_simple-28-24h         43.360834
tulipa-1_EU_investment_simple-28-4.3h      9860.583211
tulipa-1_EU_investment_simple-28-52.1h      209.519901
Name: Runtime (s), Length: 120, dtype: float64

In [ ]:
# Redo the disruptive name changes to obtain runtime from v1 results:
def disruptive_change_bench_size(name):
    if name == "temoa-utopia-1-6":
        return "temoa-utopia-1-6ts"
    if name.startswith("Sienna") and name.endswith("-1-1h"):
        return name.replace("-1-1h", "-73-1h")
    return name


total_v1_runtimes.index = total_v1_runtimes.index.map(disruptive_change_bench_size)
[b for b in s_instances if b not in set(total_v1_runtimes.index)]

[]

In [ ]:
# Use longest-processing-time-first greedy algorithm to split benchs into VMs
num_vms = 9
allocation = [[] for _ in range(num_vms)]
weights = [0 for _ in range(num_vms)]

s_instances_and_runtimes = sorted(
    [(total_v1_runtimes[b], b) for b in s_instances], reverse=True
)

for t, b in s_instances_and_runtimes:
    lightest_vm = min(enumerate(weights), key=lambda x: x[1])[0]
    allocation[lightest_vm].append(b)
    weights[lightest_vm] += t

max(weights), [len(vm) for vm in allocation], weights

(30630.100056352,
 [1, 1, 1, 1, 1, 1, 1, 6, 4],
 [30630.100056352,
  23523.15694686,
  11513.473519617999,
  8158.205818478997,
  8015.368075905006,
  7914.046962981,
  6300.398148359001,
  5590.971359530011,
  6017.366120391997])

In [ ]:
# Write allocation to infrastructure/benchmarks/ for Open Tofu:

batch_id = "20251208-test-Ss"
tfvars = f'''project_id = "compute-app-427709"
enable_gcs_upload = true
auto_destroy_vm = true
benchmarks_dir = "benchmarks/{batch_id}"
run_id = "{batch_id}"
'''

# Create a campaign folder ../infrastructure/benchmarks/{batch_id}
bench_dir = Path(f"../infrastructure/benchmarks/{batch_id}")
bench_dir.mkdir(parents=True, exist_ok=True)
with open(bench_dir / "run.tfvars", "w") as f:
    f.write(tfvars)

# Add to it the allocated benchmarks
for i, benchs in enumerate(allocation):
    y = {
        "machine-type": "c4-standard-2",  # TODO modify based on size!
        "zone": "us-central1-a",
        "years": [2020, 2022, 2023, 2024, 2025],
        # 'timeout_seconds': 24*60*60,
        "benchmarks": {},
    }
    for b in benchs:
        y["benchmarks"][s_instances[b]["bench"]] = {
            "Problem class": s_instances[b]["class"],
            "Sizes": [
                {
                    "Name": s_instances[b]["inst"],
                    "Size": s_instances[b]["size"],
                    "URL": s_instances[b]["URL"],
                }
            ],
        }
    with open(bench_dir / f"s-{i:02d}.yaml", "w") as f:
        yaml.dump(y, f, default_flow_style=False, sort_keys=False)

print(f"Created directory and files in {bench_dir}")
print(
    f"Run this campaign using the command:\ntofu apply -var-file benchmarks/{batch_id}/run.tfvars -state=states/{batch_id}.tfstate"
)

Created directory and files in ../infrastructure/benchmarks/20251208-test-Ss
Run this campaign using the command:
tofu apply -var-file benchmarks/20251208-test-Ss/run.tfvars -state=states/20251208-test-Ss.tfstate


### 20251209 Test run on all S and M instances

In [61]:
results_v1 = pd.read_csv("../results/benchmark_results.csv")

results_v1["bench-size"] = results_v1["Benchmark"] + "-" + results_v1["Size"]
results_v1["solver-version"] = results_v1["Solver"] + "-" + results_v1["Solver Version"]
results_v1.shape, len(results_v1["bench-size"].unique())

((1425, 19), 120)

In [ ]:
# Get the instances to run from the metadata file:

instances_to_run = {}
for bench_name, bench_data in metadata.get("benchmarks", {}).items():
    for size_data in bench_data.get("Sizes", []):
        if size_data["Size"] in ["S", "M"]:
            instances_to_run[bench_name + "-" + size_data["Name"]] = {
                "bench": bench_name,
                "inst": size_data["Name"],
                "class": bench_data["Problem class"],
                "size": size_data["Size"],
                "URL": size_data["URL"],
            }

# Throw out this one that no one could solve
del instances_to_run["pglib_opf_case1803_snem-1803-NA"]
# Throw out the pypsa-eur ones because new ones are coming
bench_names = list(instances_to_run.keys())
for bench_name in bench_names:
    if bench_name.startswith("pypsa-eur"):
        print("Skipping", bench_name)
        del instances_to_run[bench_name]

len(instances_to_run)

Skipping pypsa-eur-sec-2-24h
Skipping pypsa-eur-sec-6-24h
Skipping pypsa-eur-sec-5-12h
Skipping pypsa-eur-sec-2-3h
Skipping pypsa-eur-elec-trex-3-12h
Skipping pypsa-eur-elec-trex-6-12h
Skipping pypsa-eur-elec-trex-3-3h
Skipping pypsa-eur-elec-trex-4-3h
Skipping pypsa-eur-elec-op-6-24h
Skipping pypsa-eur-elec-op-8-12h
Skipping pypsa-eur-elec-op-4-3h
Skipping pypsa-eur-elec-op-2-1h
Skipping pypsa-eur-elec-op-10-3h
Skipping pypsa-eur-elec-op-ucconv-5-24h
Skipping pypsa-eur-elec-op-ucconv-2-3h
Skipping pypsa-eur-elec-op-ucconv-10-24h


88

In [62]:
# We will allocate the instances to many VMs using v1 runtimes:
total_v1_runtimes = results_v1.groupby("bench-size")["Runtime (s)"].sum()
total_v1_runtimes

bench-size
DCOPF-Carolinas_1W-1-997                    418.152434
DCOPF-Carolinas_2M-1-997                    419.125940
DCOPF-Carolinas_6M-1-997                    420.224717
DCOPF-Carolinas_uc_1W-1-997               34586.867308
DCOPF-Carolinas_uc_2M-1-997               32147.728408
                                              ...     
tulipa-1_EU_investment_simple-28-1h       54342.385765
tulipa-1_EU_investment_simple-28-2.2h     11003.532512
tulipa-1_EU_investment_simple-28-24h         43.360834
tulipa-1_EU_investment_simple-28-4.3h      9860.583211
tulipa-1_EU_investment_simple-28-52.1h      209.519901
Name: Runtime (s), Length: 120, dtype: float64

In [30]:
# Redo the disruptive name changes to obtain runtime from v1 results:
def disruptive_change_bench_size(name):
    if name.startswith("temoa"):
        return name + "ts"
    if name.startswith("Sienna") and name.endswith("-1-1h"):
        return name.replace("-1-1h", "-73-1h")
    if name.startswith("DCOPF-Carolinas") and name.endswith("-1-997"):
        return name.replace("-1-997", "-997-1h")
    changes = {
        "times-ireland-noco2-1-1h": "times-ireland-noco2-1-1ts",
        "times-ireland-noco2-40-1h": "times-ireland-noco2-40-1ts",
        "times-ireland-noco2-counties-26-1h": "times-ireland-noco2-counties-26-1ts",
        "times-nz-kea-2-24h": "times-nz-kea-2-24ts",
        "times-nz-tui-2-24h": "times-nz-tui-2-24ts",
    }
    return changes.get(name, name)


total_v1_runtimes.index = total_v1_runtimes.index.map(disruptive_change_bench_size)

missing_instances = [
    b for b in instances_to_run if b not in set(total_v1_runtimes.index)
]
if missing_instances:
    print(f"ERROR: could not find these instances in v1 runtimes: {missing_instances}")

In [ ]:
allocation, weights = allocate_vms_greedy(instances_to_run, total_v1_runtimes, 30)

Allocated. Estimated runtime: 15.5h
  VM 00: 2 instances, 15.5h
  VM 01: 2 instances, 15.3h
  VM 02: 2 instances, 15.2h
  VM 03: 2 instances, 15.2h
  VM 04: 2 instances, 15.2h
  VM 05: 3 instances, 14.9h
  VM 06: 3 instances, 14.9h
  VM 07: 3 instances, 14.9h
  VM 08: 5 instances, 14.8h
  VM 09: 3 instances, 14.9h
  VM 10: 2 instances, 15.2h
  VM 11: 3 instances, 14.9h
  VM 12: 2 instances, 15.1h
  VM 13: 4 instances, 14.9h
  VM 14: 4 instances, 14.9h
  VM 15: 4 instances, 14.8h
  VM 16: 3 instances, 14.9h
  VM 17: 4 instances, 14.9h
  VM 18: 4 instances, 14.9h
  VM 19: 3 instances, 14.9h
  VM 20: 2 instances, 14.9h
  VM 21: 3 instances, 14.9h
  VM 22: 3 instances, 14.9h
  VM 23: 3 instances, 14.9h
  VM 24: 2 instances, 14.9h
  VM 25: 2 instances, 15.1h
  VM 26: 2 instances, 15.0h
  VM 27: 3 instances, 14.9h
  VM 28: 3 instances, 14.9h
  VM 29: 5 instances, 14.9h


In [ ]:
regions_with_c4 = {
    "asia-east1-a",
    "asia-east1-c",
    "asia-northeast1-b",
    "asia-northeast1-c",
    "asia-northeast3-a",
    "asia-northeast3-b",
    "asia-northeast3-c",
    "asia-south1-a",
    "asia-south1-b",
    "asia-southeast1-a",
    "asia-southeast1-b",
    "europe-north2-a",
    "europe-north2-b",
    "europe-west1-b",
    "europe-west1-c",
    "europe-west2-a",
    "europe-west2-b",
    "europe-west3-a",
    "europe-west3-b",
    "europe-west3-c",
    "europe-west4-a",
    "europe-west4-b",
    "europe-west4-c",
    "europe-west9-a",
    "europe-west9-b",
    "northamerica-northeast2-b",
    "northamerica-northeast2-c",
    "northamerica-south1-a",
    "northamerica-south1-b",
    "northamerica-south1-c",
    "southamerica-east1-a",
    "southamerica-east1-b",
    "us-central1-a",
    "us-central1-b",
    "us-central1-c",
    "us-central1-f",
    "us-east1-b",
    "us-east1-c",
    "us-east1-d",
    "us-east4-a",
    "us-east4-b",
    "us-east4-c",
    "us-east5-a",
    "us-east5-b",
    "us-east5-c",
    "us-south1-a",
    "us-west1-a",
    "us-west3-a",
    "us-west3-b",
    "us-west4-a",
    "us-west4-b",
    "me-central2-a",
    "me-central2-c",
}
regions_by_price = [
    "europe-north2",
    "northamerica-south1",
    "us-central1",
    "us-east1",
    "us-east5",
    "us-west1",
    "asia-south1",
    "me-west1",
    "europe-west1",
    "europe-north1",
    "europe-west4",
    "northamerica-northeast1",
    "northamerica-northeast2",
    "us-west4",
    "us-east4",
    "asia-east1",
    "europe-west8",
    "europe-west9",
    "europe-southwest1",
    "us-south1",
    "asia-south2",
    "us-west2",
    "us-west3",
    "australia-southeast2",
    "me-central1",
    "asia-southeast1",
    "asia-northeast1",
    "asia-northeast2",
    "asia-northeast3",
    "europe-central2",
    "europe-west2",
    "europe-west3",
    "europe-west12",
    "africa-south1",
    "asia-southeast2",
    "europe-west6",
    "asia-east2",
    "australia-southeast1",
    "southamerica-west1",
    "europe-west10",
    "southamerica-east1",
    "me-central2",
]
regions_to_use = []
for region in regions_by_price:
    regions_to_use += [r for r in regions_with_c4 if r.startswith(region)]

# TODO I manually changed us-west1 and us-east5 because no quota / no C4s available right now

len(regions_to_use), regions_to_use[:5]

(53,
 ['europe-north2-b',
  'europe-north2-a',
  'northamerica-south1-a',
  'northamerica-south1-b',
  'northamerica-south1-c'])

In [33]:
# Write allocation to infrastructure/benchmarks/ for Open Tofu:

batch_id = "20251209-test-S-Ms"
tfvars = f'''project_id = "compute-app-427709"
enable_gcs_upload = true
auto_destroy_vm = true
benchmarks_dir = "benchmarks/{batch_id}"
run_id = "{batch_id}"
'''

# Create a campaign folder ../infrastructure/benchmarks/{batch_id}
bench_dir = Path(f"../infrastructure/benchmarks/{batch_id}")
bench_dir.mkdir(parents=True, exist_ok=True)
with open(bench_dir / "run.tfvars", "w") as f:
    f.write(tfvars)

# Add to it the allocated benchmarks
for i, benchs in enumerate(allocation):
    y = {
        "machine-type": "c4-standard-2",  # TODO modify based on size!
        "zone": regions_to_use[i],
        "years": [2020, 2022, 2023, 2024, 2025],
        # 'timeout_seconds': 24*60*60,
        "benchmarks": {},
    }
    for b in benchs:
        y["benchmarks"][instances_to_run[b]["bench"]] = {
            "Problem class": instances_to_run[b]["class"],
            "Sizes": [
                {
                    "Name": instances_to_run[b]["inst"],
                    "Size": instances_to_run[b]["size"],
                    "URL": instances_to_run[b]["URL"],
                }
            ],
        }
    with open(bench_dir / f"sm-{i:02d}.yaml", "w") as f:
        yaml.dump(y, f, default_flow_style=False, sort_keys=False)

print(f"Created directory and files in {bench_dir}")
print(
    f"Run this campaign using the command:\ntofu apply -var-file benchmarks/{batch_id}/run.tfvars -state=states/{batch_id}.tfstate"
)

Created directory and files in ../infrastructure/benchmarks/20251209-test-S-Ms
Run this campaign using the command:
tofu apply -var-file benchmarks/20251209-test-S-Ms/run.tfvars -state=states/20251209-test-S-Ms.tfstate


### Find low runtime variance GCP zones
TODO delete? We're just using us-central1 for now?


In [ ]:
_, var_so_far = load_results(
    [
        "../results/gcp-results/20251128-test-hipo/",
        "../results/gcp-results/20251128-test-hipo-alpha/",
        "../results/gcp-results/20251201-hipo-new-pypsa/",
        "../results/gcp-results/20251202-test-PRs/",
        "../results/gcp-results/20251203-test-PRs/",
        "../results/gcp-results/20251207-test-PRs/",
        "../results/gcp-results/20251207-test-PRs-2/",
        "../results/gcp-results/20251208-test-solvers/",
        "../results/gcp-results/20251208-test-solvers-2/",
        "../results/gcp-results/20251208-test-Ss/",
        "../results/gcp-results/20251209-test-pypsa-PRs/",
        "../results/gcp-results/20251209-test-S-Ms/",
        "../results/gcp-results/20251211-test-pypsa-168-switch-usa/",
        "../results/gcp-results/20251212-run-Ls/",
        "../results/gcp-results/20251212-test-crossover/",
        "../results/gcp-results/20251213-run-S-M/",
        "../results/gcp-results/20251214-rerun-1/",
        "../results/gcp-results/20251215-run-S-M/",
        "../results/gcp-results/20251216-rerun-2/",
        "../results/gcp-results/20251216-test-PRs/",
        "../results/gcp-results/20251219-pypsa-sizes/",
        "../results/gcp-results/20251220-iesa/",
    ]
)
var_so_far.sort_values("std %", ascending=False)

Found 4513 records, 333 benchmark instances


/Users/sid/code/solver-benchmark/runner/utils.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)


Runtime (s)  \
                                                                                      count   
Hostname                             Run ID               VM Zone                             
benchmark-instance-sm-03             20251209-test-S-Ms   northamerica-south1-b          16   
benchmark-instance-sm-10             20251209-test-S-Ms   us-east1-c                     15   
benchmark-instance-01-pypsa-at       20251202-test-PRs    us-central1-a                   2   
benchmark-instance-l-01              20251212-run-Ls      us-central1-b                   2   
benchmark-instance-sm-26             20251209-test-S-Ms   us-west4-a                     16   
...                                                                                     ...   
benchmark-instance-pypsa-sizes-08    20251219-pypsa-sizes us-central1-a                   1   
benchmark-instance-pypsa-sizes-09    20251219-pypsa-sizes us-central1-a                   1   
benchmark-instance-pypsa-sizes-10    20251219-pypsa-sizes us-central1-a                   1   
benchmark-instance-test-fine         20251216-test-PRs    us-central1-a                   1   
benchmark-instance-test-switch-china 20251216-test-PRs    us-central1-a                   1   

                                                                                             \
                                                                                        min   
Hostname                             Run ID               VM Zone                             
benchmark-instance-sm-03             20251209-test-S-Ms   northamerica-south1-b  179.100682   
benchmark-instance-sm-10             20251209-test-S-Ms   us-east1-c             178.880643   
benchmark-instance-01-pypsa-at       20251202-test-PRs    us-central1-a          183.987588   
benchmark-instance-l-01              20251212-run-Ls      us-central1-b          190.303724   
benchmark-instance-sm-26             20251209-test-S-Ms   us-west4-a             185.017391   
...                                                                                     ...   
benchmark-instance-pypsa-sizes-08    20251219-pypsa-sizes us-central1-a          223.262342   
benchmark-instance-pypsa-sizes-09    20251219-pypsa-sizes us-central1-a          221.899480   
benchmark-instance-pypsa-sizes-10    20251219-pypsa-sizes us-central1-a          227.473434   
benchmark-instance-test-fine         20251216-test-PRs    us-central1-a          230.626723   
benchmark-instance-test-switch-china 20251216-test-PRs    us-central1-a          228.028572   

                                                                                             \
                                                                                        max   
Hostname                             Run ID               VM Zone                             
benchmark-instance-sm-03             20251209-test-S-Ms   northamerica-south1-b  241.477689   
benchmark-instance-sm-10             20251209-test-S-Ms   us-east1-c             242.037828   
benchmark-instance-01-pypsa-at       20251202-test-PRs    us-central1-a          213.681669   
benchmark-instance-l-01              20251212-run-Ls      us-central1-b          211.791369   
benchmark-instance-sm-26             20251209-test-S-Ms   us-west4-a             216.704818   
...                                                                                     ...   
benchmark-instance-pypsa-sizes-08    20251219-pypsa-sizes us-central1-a          223.262342   
benchmark-instance-pypsa-sizes-09    20251219-pypsa-sizes us-central1-a          221.899480   
benchmark-instance-pypsa-sizes-10    20251219-pypsa-sizes us-central1-a          227.473434   
benchmark-instance-test-fine         20251216-test-PRs    us-central1-a          230.626723   
benchmark-instance-test-switch-china 20251216-test-PRs    us-central1-a          228.028572   

                                                                                       

In [ ]:
var_by_zone = {}
for i, r in var_so_far["std %"].items():
    var_by_zone.setdefault(i[2], []).append(r)
sorted(var_by_zone["us-central1-a"])

[0.19569016852889487,
 1.5280941600769635,
 nan,
 0.5525105172024867,
 10.55997448128975,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 4.652784178425014,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.24157329062480262,
 nan,
 0.07226069993916358,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.38857742111072585,
 nan,
 nan,
 0.3437866541064088,
 0.40163628125704565,
 0.6185229545159973,
 0.8070416992136743,
 0.8893440141241269,
 1.0157248365479818,
 1.267759921165408,
 1.4531646371849027,
 1.721324418947807,
 1.8371093380952683,
 1.8712246104136268,
 2.078889367741032,
 2.2377760958165327,
 nan,
 nan,
 nan,
 2.3260139396206085,
 2.699162036010941,
 2.849257172885964,
 5.293286580746238,
 6.5187562550573155,
 nan,
 nan]

### Test zone pricing script

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/sid/code/solver-benchmark/orchestrator-gcp-key.json"
)
%run ../runner/c4_machine_info.py

region,price_hour,zones
africa-south1,0.126701,africa-south1-a africa-south1-b africa-south1-c
asia-east1,0.112161,asia-east1-a asia-east1-c
asia-east2,0.135506,asia-east2-a asia-east2-c
asia-northeast1,0.124395,asia-northeast1-b asia-northeast1-c
asia-northeast2,0.124395,asia-northeast2-a asia-northeast2-c
asia-northeast3,0.124395,asia-northeast3-a asia-northeast3-b asia-northeast3-c
asia-south1,0.100741,asia-south1-a asia-south1-b asia-south1-c
asia-south2,0.100744,asia-south2-a asia-south2-b asia-south2-c
asia-southeast1,0.119504,asia-southeast1-a asia-southeast1-b asia-southeast1-c
asia-southeast2,0.130256,asia-southeast2-a asia-southeast2-b
asia-southeast3,0.101705,
australia-southeast1,0.121082,australia-southeast1-c
australia-southeast2,0.125926,australia-southeast2-a australia-southeast2-b australia-southeast2-c
europe-central2,0.117202,europe-central2-a europe-central2-c
europe-north1,0.106659,europe-north1-a europe-north1-c
europe-north2,0.101709,europe-north2-a europe-north2

### 20251212 Run smallest Ls on cheapest zones

In [104]:
# Run all v1 Ls, don't run new pypsa-eur, run smallest 1h pypsa-de
# TODO continue
bench_set = set()
for b, r in benchmarks_df.iterrows():
    if r["Size"] != "L" or b.startswith("pypsa-eur"):
        continue
    if b.startswith("pypsa-de") and r["Instance"] != "50-1h":
        continue
    bench_set.add(b)

benchs_to_run = benchmarks_df[benchmarks_df.index.isin(bench_set)].copy()
benchs_to_run[["Size", "Num. variables", "v1 runtime"]]

,Size,Num. variables,v1 runtime
genx-elec_trex-15-168h,L,13777226.0,79441.437168
genx-elec_trex_co2-15-168h,L,13777227.0,87509.210944
genx-elec_trex_uc-15-24h,L,2551346.0,36850.939651
genx-elec_co2-15-168h,L,13777201.0,80570.337039
tulipa-1_EU_investment_simple-28-1h,L,1619451.0,54342.385765
pypsa-de-elec-50-1h,L,8637885.0,NaN
pypsa-de-elec-trex_vopt-50-1h,L,8638003.0,NaN
pypsa-de-elec-trex_copt-50-1h,L,8638003.0,NaN
pypsa-de-elec-dfp-50-1h,L,8637885.0,NaN
pypsa-de-elec-trex_vopt-dfp-50-1h,L,8638003.0,NaN


In [96]:
# Find cheapest zones within variability threshold

variability_threshold = 4.0
zones = []

region_prices = pd.read_csv("gcp-region-prices.csv").sort_values("price_hour")
for _, r in region_prices[~region_prices["zones"].isna()].iterrows():
    for z in r["zones"].split():
        # TODO also skip us-west1 -- no quota there! No c4-highmem in asia-south2-c & us-east5-c & europe-north1-c
        if max(var_by_zone.get(z, [0.0])) < variability_threshold:
            zones.append((z, r["price_hour"]))
print(
    f"Found {len(zones)} zones not yet exceeding threshold of {variability_threshold}%"
)

Found 85 zones not yet exceeding threshold of 4.0%


In [105]:
# Allocate to cheapest zones within variability cutoff

# TODO maybe put larger ones on cheaper zones?

zone_per_bench = {}
price_per_bench = {}
for i, b in enumerate(benchs_to_run.index):
    zone_per_bench[b] = zones[i][0]
    price_per_bench[b] = zones[i][1]
benchs_to_run["zone"] = zone_per_bench
benchs_to_run["price"] = price_per_bench

print(
    "Benchmarks allocated to zones with prices between",
    min(benchs_to_run["price"]),
    max(benchs_to_run["price"]),
)

benchs_to_run

Benchmarks allocated to zones with prices between 0.096866 0.106553


,Benchmark,Instance,Modelling framework,Model name,Version,Contributor(s)/Source,License,Problem class,Application,Sectoral focus,...,Realistic,Num. constraints,Num. variables,Num. continuous variables,Num. integer variables,spatial resolution,Notes,v1 runtime,zone,price
genx-elec_trex-15-168h,genx-elec_trex,15-168h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,11832658.0,13777226.0,NaN,NaN,NaN,NaN,79441.437168,us-central1-a,0.096866
genx-elec_trex_co2-15-168h,genx-elec_trex_co2,15-168h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,11832689.0,13777227.0,NaN,NaN,NaN,NaN,87509.210944,us-central1-b,0.096866
genx-elec_trex_uc-15-24h,genx-elec_trex_uc,15-24h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,MILP,Infrastructure & Capacity Expansion,Power-only,...,True,3123898.0,2551346.0,2062938.0,488408.0,NaN,NaN,36850.939651,us-central1-f,0.096866
genx-elec_co2-15-168h,genx-elec_co2,15-168h,GenX,GenX,None,"Gabe Mantegna, Princeton University; Daniele L...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,11832633.0,13777201.0,NaN,NaN,NaN,NaN,80570.337039,us-east4-a,0.096866
tulipa-1_EU_investment_simple-28-1h,tulipa-1_EU_investment_simple,28-1h,Tulipa,Tulipa,None,https://github.com/jump-dev/open-energy-modeli...,"Model code licensed MIT, benchmark instances n...",MILP,Infrastructure & Capacity expansion,Power-only,...,True,2429440.0,1619451.0,1619199.0,252.0,NaN,NaN,54342.385765,us-east4-c,0.096866
pypsa-de-elec-50-1h,pypsa-de-elec,50-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,18370417.0,8637885.0,NaN,NaN,NaN,NaN,NaN,us-east5-a,0.096866
pypsa-de-elec-trex_vopt-50-1h,pypsa-de-elec-trex_vopt,50-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,18370653.0,8638003.0,NaN,NaN,NaN,NaN,NaN,us-east5-b,0.096866
pypsa-de-elec-trex_copt-50-1h,pypsa-de-elec-trex_copt,50-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,18370653.0,8638003.0,NaN,NaN,NaN,NaN,NaN,us-east5-c,0.096866
pypsa-de-elec-dfp-50-1h,pypsa-de-elec-dfp,50-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,18370417.0,8637885.0,NaN,NaN,NaN,NaN,NaN,us-west1-a,0.096866
pypsa-de-elec-trex_vopt-dfp-50-1h,pypsa-de-elec-trex_vopt-dfp,50-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,True,18370653.0,8638003.0,NaN,NaN,NaN,NaN,NaN,us-west1-b,0.096866


In [107]:
# Create campaign:
from pathlib import Path

batch_id = "20251212-run-Ls"
tfvars = f'''project_id = "compute-app-427709"
enable_gcs_upload = true
auto_destroy_vm = true
benchmarks_dir = "benchmarks/{batch_id}"
run_id = "{batch_id}"
'''

# Create a campaign folder ../infrastructure/benchmarks/{batch_id}
bench_dir = Path(f"../infrastructure/benchmarks/{batch_id}")
bench_dir.mkdir(parents=True, exist_ok=True)
with open(bench_dir / "run.tfvars", "w") as f:
    f.write(tfvars)

# Add to it the allocated benchmarks
idx = 0
for _, benchmark in benchs_to_run.iterrows():
    y = {
        "machine-type": "c4-highmem-16",  # TODO get_machine_type(bench['size']),
        "zone": benchmark["zone"],
        "years": [2025, 2024],
        "timeout_seconds": 24 * 60 * 60,
        "benchmarks": {
            benchmark["Benchmark"]: {
                "Problem class": benchmark["Problem class"],
                "Sizes": [
                    {
                        "Name": benchmark["Instance"],
                        "Size": benchmark["Size"],
                        "URL": benchmark["URL"],
                    }
                ],
            }
        },
    }
    with open(bench_dir / f"l-{idx:02d}.yaml", "w") as f:
        yaml.dump(y, f, default_flow_style=False, sort_keys=False)
    idx += 1

print(f"Created directory and files in {bench_dir}")
print(
    f"Run this campaign from the infrastructure/ directory using the command:\ntofu apply -var-file benchmarks/{batch_id}/run.tfvars -state=states/{batch_id}.tfstate"
)

Created directory and files in ../infrastructure/benchmarks/20251212-run-Ls
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20251212-run-Ls/run.tfvars -state=states/20251212-run-Ls.tfstate


### 20251213 Run the Ss & Ms
Some VMs from `20251209-test-S-Ms` had large variability. Also now we are running HiGHS variants with `run_crossover=choose`.

In [128]:
# For weights use 1209 runtimes, or v1 runtime, or average runtime
results_1209, variability_1209 = load_results(
    "../results/gcp-results/20251209-test-S-Ms/"
)
benchmarks_df["est. runtime"] = results_1209.groupby("bench-size")["Runtime (s)"].sum()

# Check that all bench-sizes have the same number of results (solvers)
counts = results_1209.groupby("bench-size").size()
num_solvers_v2 = max(counts)

# Fill NA est. runtime with v1 runtime scaled by the ratio of num solvers
benchmarks_df.loc[benchmarks_df["est. runtime"].isna(), "est. runtime"] = (
    benchmarks_df["v1 runtime"] * num_solvers_v2 / 13
)

# Fill remaining NA with average of non-NA est. runtime
avg_runtime = benchmarks_df["est. runtime"].dropna().mean()
benchmarks_df["est. runtime"] = benchmarks_df["est. runtime"].fillna(avg_runtime)

Found 1357 records, 87 benchmark instances


In [131]:
# Run all S & M, except pypsa-eur
bench_set = {
    b
    for b, r in benchmarks_df.iterrows()
    if not (r["Size"] == "L" or b.startswith("pypsa-eur") or b.startswith("pypsa-de"))
}

benchs_to_run = benchmarks_df[benchmarks_df.index.isin(bench_set)].copy()
benchs_to_run[["Size", "Num. variables", "est. runtime"]]

,Size,Num. variables,est. runtime
genx-1_three_zones-no_uc-3-1h,M,103506.0,6835.319799
genx-2_three_zones_w_electrolyzer-no_uc-3-1h,M,153413.0,5422.105690
genx-3_three_zones_w_co2_capture-no_uc-3-1h,M,103506.0,1173.016479
genx-4_three_zones_w_policies_slack-no_uc-3-1h,M,121992.0,11965.652900
genx-6_three_zones_w_multistage-no_uc-3-1h,M,103612.0,6406.168654
...,...,...,...
times-etimeseu-europe-elec+heat-single_stage-29-64ts,M,452854.0,2179.191305
times-etimeseu-europe-elec+heat-co2-single_stage-29-64ts,M,452914.0,2429.734196
times-ireland-noco2-1-1ts,M,544694.0,33122.137988
times-nz-kea-2-24ts,M,202797.0,15242.495821


In [161]:
# Allocate greedily to 13 VMs:
vm_yamls = allocate_benchmarks(benchs_to_run, "est. runtime", 15)

Allocated. Estimated runtime: 46.2h
  VM 00: 6 instances, 45.8h
  VM 01: 7 instances, 45.8h
  VM 02: 7 instances, 45.8h
  VM 03: 6 instances, 45.7h
  VM 04: 8 instances, 45.8h
  VM 05: 7 instances, 45.8h
  VM 06: 8 instances, 45.8h
  VM 07: 7 instances, 45.8h
  VM 08: 8 instances, 45.8h
  VM 09: 8 instances, 45.7h
  VM 10: 6 instances, 46.1h
  VM 11: 8 instances, 45.8h
  VM 12: 7 instances, 45.7h
  VM 13: 11 instances, 45.7h
  VM 14: 6 instances, 46.2h


In [162]:
# Find cheapest zones within variability threshold

variability_threshold = 4.0
zones = []
avoid_zones = [
    "us-west1",
    "us-west2",
    "asia-east1",
]  # TODO No c4-highmem in asia-south2-c & us-east5-c & europe-north1-c

region_prices = pd.read_csv("gcp-region-prices.csv").sort_values("price_hour")
for _, r in region_prices[~region_prices["zones"].isna()].iterrows():
    for z in r["zones"].split():
        if any(z.startswith(p) for p in avoid_zones):
            continue
        if max(var_by_zone.get(z, [0.0])) < variability_threshold:
            zones.append((z, r["price_hour"]))
print(
    f"Found {len(zones)} zones not yet exceeding threshold of {variability_threshold}%"
)

Found 78 zones not yet exceeding threshold of 4.0%


In [163]:
# Allocate to cheapest zones within variability cutoff
# TODO maybe put larger ones on cheaper zones?

# Skip the first 30 because we're already using them
zones = zones[30:]

prices_used = []
for i, y in enumerate(vm_yamls):
    y["zone"] = zones[i][0]
    prices_used.append(zones[i][1])

print(
    "Benchmarks allocated to zones with prices between",
    min(prices_used),
    max(prices_used),
)

Benchmarks allocated to zones with prices between 0.110427 0.116443


In [164]:
# Create campaign:
from pathlib import Path

batch_id = "20251213-run-S-M"
vm_prefix = "s-m"
tfvars = f'''project_id = "compute-app-427709"
enable_gcs_upload = true
auto_destroy_vm = true
benchmarks_dir = "benchmarks/{batch_id}"
run_id = "{batch_id}"
'''

# Create a campaign folder ../infrastructure/benchmarks/{batch_id}
bench_dir = Path(f"../infrastructure/benchmarks/{batch_id}")
bench_dir.mkdir(parents=True, exist_ok=True)
with open(bench_dir / "run.tfvars", "w") as f:
    f.write(tfvars)

# Add to it the allocated benchmarks
for idx, yaml_data in enumerate(vm_yamls):
    with open(bench_dir / f"{vm_prefix}-{idx:02d}.yaml", "w") as f:
        yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

print(f"Created directory and files in {bench_dir}")
print(
    f"Run this campaign from the infrastructure/ directory using the command:\ntofu apply -var-file benchmarks/{batch_id}/run.tfvars -state=states/{batch_id}.tfstate"
)

Created directory and files in ../infrastructure/benchmarks/20251213-run-S-M
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20251213-run-S-M/run.tfvars -state=states/20251213-run-S-M.tfstate


### 20251215 Re-run the Ss & Ms
The `20251213` run unfortunately used the default OpenTofu `years` which only ran 2024! So I manually copied the folder and ran this command to update the yaml files with the list of all years:

In [196]:
for f in Path("../infrastructure/benchmarks/20251215-run-S-M/").glob("*.yaml"):
    contents = open(f, "r").read()
    contents = contents.replace(
        "benchmarks:", "years:\n- 2020\n- 2022\n- 2023\n- 2024\n- 2025\nbenchmarks:"
    )
    with open(f, "w") as f:
        f.write(contents)

### 20251216 Re-run bad variability VMs and new Switch-CN
Manually changed the zones to low-variability ones. TODO automate this?

### 20251222 Run new & leftover instances
After merging many new contribution PRs


In [51]:
ran_so_far, _ = load_results(
    [
        "../results/gcp-results/20251212-run-Ls/",
        "../results/gcp-results/20251214-rerun-1/",
        "../results/gcp-results/20251215-run-S-M/",
        "../results/gcp-results/20251216-rerun-2/",
    ]
)

# Correct the bug in allocate_benchmarks where bench-size looks like bench-size-size
ran_so_far["Benchmark"] = ran_so_far.apply(
    lambda row: row["Benchmark"][: -len(row["Size"]) - 1]
    if row["Benchmark"].endswith(row["Size"])
    else row["Benchmark"],
    axis=1,
)
ran_so_far["bench-size"] = ran_so_far["Benchmark"] + "-" + ran_so_far["Size"]

benchs_run_so_far = set(ran_so_far["bench-size"].unique())

benchmarks_df = load_benchmark_metadata()
benchs_available = set(benchmarks_df.index)
benchs_to_run = benchs_available - benchs_run_so_far

# Remove pypsa-eur/de, this will be run later
benchs_to_run = {
    b
    for b in benchs_to_run
    if not (b.startswith("pypsa-eur") or b.startswith("pypsa-de"))
}
benchs_to_run = benchmarks_df[benchmarks_df.index.isin(benchs_to_run)].copy()

# Remove those marked Skip
benchs_to_run = benchs_to_run[benchs_to_run["Skip because"].isna()]

benchs_to_run[["Size", "Num. variables"]]

Found 2100 records, 129 benchmark instances


,Size,Num. variables
ethos-fine-multi-regional-7tp-8-168ts,M,55979.0
ethos-fine-multi-regional-7tp-12seg-8-84ts,M,33215.0
ethos-fine-energyland-full-timeseries-1-8760ts,M,473179.0
ethos-fine-energyland-48tp-1-1152ts,M,32770.0
FINE-2-node-electricity-supply-system-2-8760ts,S,8152.0
FINE-1-node-energy-system-workflow-1-8760ts,S,4892.0
FINE-energyland-1-8760ts,M,32770.0
FINE-multi-regional-energy-system-workflow-8-8760ts,M,16580.0
FINE-district-optimization-14-8760ts,M,15260.0
FINE-water-supply-system-12-8760ts,S,4701.0


In [54]:
# Create benchmark campaign
vm_yamls = allocate_benchmarks(
    benchs_to_run.query('Size == "L"'),
    "Num. variables",
    1,
    machine_type="c4-highmem-16",
    timeout_seconds=24 * 60 * 60,
)
vm_yamls += allocate_benchmarks(
    benchs_to_run.query('Size != "L"'), "Num. variables", 10
)

create_benchmark_campaign("20251222-leftovers", "leftovers", vm_yamls)

Allocated. Estimated runtime: 862.6h
  VM 00: 1 instances, 862.6h
Allocated. Estimated runtime: 131.4h
  VM 00: 1 instances, 131.4h
  VM 01: 1 instances, 25.5h
  VM 02: 1 instances, 24.3h
  VM 03: 1 instances, 17.0h
  VM 04: 1 instances, 15.5h
  VM 05: 3 instances, 11.3h
  VM 06: 2 instances, 11.4h
  VM 07: 2 instances, 11.3h
  VM 08: 4 instances, 11.5h
  VM 09: 4 instances, 11.0h
Created directory and files in ../infrastructure/benchmarks/20251222-leftovers
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20251222-leftovers/run.tfvars -state=states/20251222-leftovers.tfstate


### 20251227 Run new pypsa benchmarks
Running the sizes of pypsa-de-elec that are likely to be solvable based on the experiment in ``, and all sizes of pypsa-eur-elec (<10m vars) to see what might be solvable for its variants. Skipping pyspa-de-sec* for now because the smallest Ls timeout with even Gurobi.


In [87]:
to_run = {
    "pypsa-de-elec": ["10-3h", "10-1h", "50-3h", "20-1h"],
    "pypsa-de-elec-dfp": ["10-3h", "10-1h", "50-3h", "20-1h"],
    "pypsa-de-elec-trex_copt": ["10-3h", "10-1h", "50-3h", "20-1h"],
    "pypsa-de-elec-trex_copt-dfp": ["10-3h", "10-1h", "50-3h", "20-1h"],
    "pypsa-de-elec-trex_vopt": ["10-3h", "10-1h", "50-3h", "20-1h"],
    "pypsa-de-elec-trex_vopt-dfp": ["10-3h", "10-1h", "50-3h", "20-1h"],
    # Running all reasonable sizes of pypsa-eur-elec to test
    "pypsa-eur-elec": ["50-24h", "100-24h", "50-12h", "100-12h", "50-3h", "100-3h"],
    # Skipping pypsa-de-sec* for now because the smallest Ls timeout with even Gurobi
}
benchs_to_run = set()
for b, sizes in to_run.items():
    for s in sizes:
        benchs_to_run.add(b + "-" + s)

benchmarks_df = load_benchmark_metadata()
benchs_to_run = benchmarks_df[benchmarks_df.index.isin(benchs_to_run)].copy()
benchs_to_run

,Benchmark,Instance,Modelling framework,Model name,Version,Contributor(s)/Source,License,Problem class,Application,Sectoral focus,...,URL,Temporal resolution,Spatial resolution,Realistic,Num. constraints,Num. variables,Skip because,Num. continuous variables,Num. integer variables,Notes
pypsa-eur-elec-50-3h,pypsa-eur-elec,50-3h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,3 hours,50 nodes,True,7379675.0,3542548.0,NaN,NaN,NaN,NaN
pypsa-eur-elec-50-12h,pypsa-eur-elec,50-12h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,50 nodes,False,1845545.0,886078.0,NaN,NaN,NaN,NaN
pypsa-eur-elec-50-24h,pypsa-eur-elec,50-24h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,24 hours,50 nodes,False,923190.0,443333.0,NaN,NaN,NaN,NaN
pypsa-eur-elec-100-3h,pypsa-eur-elec,100-3h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,3 hours,100 nodes,True,14064375.0,6720092.0,NaN,NaN,NaN,NaN
pypsa-eur-elec-100-12h,pypsa-eur-elec,100-12h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,100 nodes,True,3517335.0,1680902.0,NaN,NaN,NaN,NaN
pypsa-eur-elec-100-24h,pypsa-eur-elec,100-24h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,24 hours,100 nodes,False,1759495.0,841037.0,NaN,NaN,NaN,NaN
pypsa-de-elec-10-1h,pypsa-de-elec,10-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,1 hour,10 nodes,False,4502789.0,2111271.0,NaN,NaN,NaN,NaN
pypsa-de-elec-10-3h,pypsa-de-elec,10-3h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,3 hours,10 nodes,False,1501029.0,703831.0,NaN,NaN,NaN,NaN
pypsa-de-elec-20-1h,pypsa-de-elec,20-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,1 hour,20 nodes,True,8208403.0,3854613.0,NaN,NaN,NaN,NaN
pypsa-de-elec-50-3h,pypsa-de-elec,50-3h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,3 hours,50 nodes,True,6123937.0,2879645.0,NaN,NaN,NaN,NaN


In [92]:
# Create benchmark campaign
l_to_run = benchs_to_run.query('Size == "L"')
vm_yamls = allocate_benchmarks(
    l_to_run,
    "Num. variables",
    len(l_to_run),
    machine_type="c4-highmem-16",
    timeout_seconds=24 * 60 * 60,
)
vm_yamls += allocate_benchmarks(benchs_to_run.query('Size != "L"'), "Num. variables", 9)

create_benchmark_campaign("20251227-new-pypsa", "new-pypsa", vm_yamls)

Allocated. Estimated runtime: 1866.7h
  VM 00: 1 instances, 1866.7h
  VM 01: 1 instances, 1070.7h
  VM 02: 1 instances, 1070.7h
  VM 03: 1 instances, 1070.7h
  VM 04: 1 instances, 1070.7h
  VM 05: 1 instances, 1070.7h
  VM 06: 1 instances, 1070.7h
  VM 07: 1 instances, 984.0h
  VM 08: 1 instances, 799.9h
  VM 09: 1 instances, 799.9h
  VM 10: 1 instances, 799.9h
  VM 11: 1 instances, 799.9h
  VM 12: 1 instances, 799.9h
  VM 13: 1 instances, 799.9h
  VM 14: 1 instances, 586.5h
  VM 15: 1 instances, 586.5h
  VM 16: 1 instances, 586.5h
  VM 17: 1 instances, 586.5h
  VM 18: 1 instances, 586.5h
  VM 19: 1 instances, 586.5h
  VM 20: 1 instances, 466.9h
Allocated. Estimated runtime: 246.1h
  VM 00: 1 instances, 246.1h
  VM 01: 1 instances, 233.6h
  VM 02: 1 instances, 195.5h
  VM 03: 1 instances, 195.5h
  VM 04: 1 instances, 195.5h
  VM 05: 1 instances, 195.5h
  VM 06: 1 instances, 195.5h
  VM 07: 1 instances, 195.5h
  VM 08: 1 instances, 123.1h
Created directory and files in ../infrastructure

**Note**: reached 300 CPU limit in us-central1-a, so manually adjusted some yaml files to other zones and re-launched

In [94]:
region_prices = pd.read_csv("gcp-region-prices.csv").sort_values("price_hour")
region_prices

,region,price_hour,zones
0,us-central1,0.096866,us-central1-a us-central1-b us-central1-c us-c...
1,us-east1,0.096866,us-east1-b us-east1-c us-east1-d
2,us-east4,0.096866,us-east4-a us-east4-b us-east4-c
3,us-east5,0.096866,us-east5-a us-east5-b us-east5-c
4,us-west1,0.096866,us-west1-a us-west1-b us-west1-c
5,asia-south1,0.100741,asia-south1-a asia-south1-b asia-south1-c
6,asia-south2,0.100744,asia-south2-a asia-south2-b asia-south2-c
7,asia-southeast3,0.101705,NaN
9,europe-west4,0.101709,europe-west4-a europe-west4-b europe-west4-c
8,europe-north2,0.101709,europe-north2-a europe-north2-b europe-north2-c


In [107]:
# Check in-progress results for runtime variability, in case something needs to be killed and re-run

# fetch_all_partial_results()

partial_results, partial_variability = load_results(
    ["../results/gcp-results/20251227-new-pypsa/", "../results/partial-results/"]
)

partial_variability.sort_values("std %", ascending=False)

Found 153 records, 9 benchmark instances


/Users/sid/code/solver-benchmark/runner/utils.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)


Runtime (s)  \
                                                                       count   
Hostname                        Run ID             VM Zone                     
benchmark-instance-new-pypsa-21 20251227-new-pypsa us-central1-a          11   
benchmark-instance-new-pypsa-23 20251227-new-pypsa us-central1-a          17   
benchmark-instance-new-pypsa-28 20251227-new-pypsa us-central1-a          12   
benchmark-instance-new-pypsa-26 20251227-new-pypsa us-central1-a          16   
benchmark-instance-new-pypsa-27 20251227-new-pypsa us-central1-a          15   
benchmark-instance-new-pypsa-29 20251227-new-pypsa us-central1-a          15   
benchmark-instance-new-pypsa-24 20251227-new-pypsa us-central1-a          15   
benchmark-instance-new-pypsa-22 20251227-new-pypsa us-central1-a          11   
benchmark-instance-new-pypsa-25 20251227-new-pypsa us-central1-a          16   

                                                                              \
                                                                         min   
Hostname                        Run ID             VM Zone                     
benchmark-instance-new-pypsa-21 20251227-new-pypsa us-central1-a  230.004267   
benchmark-instance-new-pypsa-23 20251227-new-pypsa us-central1-a  207.819509   
benchmark-instance-new-pypsa-28 20251227-new-pypsa us-central1-a  212.938169   
benchmark-instance-new-pypsa-26 20251227-new-pypsa us-central1-a  215.262283   
benchmark-instance-new-pypsa-27 20251227-new-pypsa us-central1-a  217.559635   
benchmark-instance-new-pypsa-29 20251227-new-pypsa us-central1-a  221.608072   
benchmark-instance-new-pypsa-24 20251227-new-pypsa us-central1-a  217.170568   
benchmark-instance-new-pypsa-22 20251227-new-pypsa us-central1-a  218.371302   
benchmark-instance-new-pypsa-25 20251227-new-pypsa us-central1-a  213.314904   

                                                                              \
                                                                         max   
Hostname                        Run ID             VM Zone                     
benchmark-instance-new-pypsa-21 20251227-new-pypsa us-central1-a  283.976696   
benchmark-instance-new-pypsa-23 20251227-new-pypsa us-central1-a  227.140023   
benchmark-instance-new-pypsa-28 20251227-new-pypsa us-central1-a  228.659666   
benchmark-instance-new-pypsa-26 20251227-new-pypsa us-central1-a  231.258771   
benchmark-instance-new-pypsa-27 20251227-new-pypsa us-central1-a  229.789631   
benchmark-instance-new-pypsa-29 20251227-new-pypsa us-central1-a  232.924509   
benchmark-instance-new-pypsa-24 20251227-new-pypsa us-central1-a  229.937981   
benchmark-instance-new-pypsa-22 20251227-new-pypsa us-central1-a  230.740854   
benchmark-instance-new-pypsa-25 20251227-new-pypsa us-central1-a  221.300018   

                                                                             \
                                                                        std   
Hostname                        Run ID             VM Zone                    
benchmark-instance-new-pypsa-21 20251227-new-pypsa us-central1-a  19.189780   
benchmark-instance-new-pypsa-23 20251227-new-pypsa us-central1-a   5.215577   
benchmark-instance-new-pypsa-28 20251227-new-pypsa us-central1-a   5.195654   
benchmark-instance-new-pypsa-26 20251227-new-pypsa us-central1-a   4.257744   
benchmark-instance-new-pypsa-27 20251227-new-pypsa us-central1-a   4.179907   
benchmark-instance-new-pypsa-29 20251227-new-pypsa us-central1-a   3.825181   
benchmark-instance-new-pypsa-24 20251227-new-pypsa us-central1-a   3.712972   
benchmark-instance-new-pypsa-22 20251227-new-pypsa us-central1-a   3.743062   
benchmark-instance-new-pypsa-25 20251227-new-pypsa us-central1-a   2.110697   

                                                                              \
                                                                        mean   
Hostname                        Run ID             VM Zone               

new-pypsa-21 has pypsa-eur-elec, which we were mostly running to see which sizes to keep, so leave it in for now. Can be re-run later if we want the results for website

### 20260102 Run more new pypsa benchmarks


In [160]:
to_run = {
    "pypsa-de-elec-dfp": ["50-24h"],
    "pypsa-de-elec-trex_copt": ["50-24h", "20-3h"],
    "pypsa-de-elec-trex_copt-dfp": ["50-24h"],
    "pypsa-de-elec-trex_vopt": ["50-24h", "20-3h"],
    "pypsa-de-elec-trex_vopt-dfp": ["50-24h", "20-3h"],
    "pypsa-de-sec": ["10-168h", "50-168h", "20-12h"],
    "pypsa-de-sec-trex_copt": ["50-168h", "10-24h", "20-12h"],
    "pypsa-de-sec-trex_vopt": ["50-168h", "10-24h", "20-12h"],
    "pypsa-eur-elec": ["50-168h"],
    "pypsa-eur-elec-dfp": ["50-168h", "100-12h"],
    "pypsa-eur-elec-trex_copt": ["50-168h", "100-12h"],
    "pypsa-eur-elec-trex_copt-dfp": ["50-168h", "100-12h"],
    "pypsa-eur-elec-trex_vopt": ["50-168h", "100-12h"],
    "pypsa-eur-elec-trex_vopt-dfp": ["50-168h", "100-12h"],
    "pypsa-eur-sec": ["50-168h", "50-24h"],
    "pypsa-eur-sec-trex_copt": ["50-168h", "50-24h"],
    "pypsa-eur-sec-trex_vopt": ["50-168h", "50-24h"],
}
benchs_to_run = set()
for b, sizes in to_run.items():
    for s in sizes:
        benchs_to_run.add(b + "-" + s)

benchmarks_df = load_benchmark_metadata()
benchs_to_run = benchmarks_df[benchmarks_df.index.isin(benchs_to_run)].copy()
benchs_to_run

,Benchmark,Instance,Modelling framework,Model name,Version,Contributor(s)/Source,License,Problem class,Application,Sectoral focus,...,URL,Temporal resolution,Spatial resolution,Realistic,Num. constraints,Num. variables,Skip because,Num. continuous variables,Num. integer variables,Notes
pypsa-eur-elec-50-168h,pypsa-eur-elec,50-168h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,168 hours,50 nodes,False,134766,64877,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_vopt-50-168h,pypsa-eur-elec-trex_vopt,50-168h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,168 hours,50 nodes,False,135086,65037,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_vopt-100-12h,pypsa-eur-elec-trex_vopt,100-12h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,100 nodes,True,3517891,1681180,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_copt-50-168h,pypsa-eur-elec-trex_copt,50-168h,PyPSA,PyPSA-Eur,<insert version>,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,168 hours,50 nodes,False,135086,65037,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_copt-100-12h,pypsa-eur-elec-trex_copt,100-12h,PyPSA,PyPSA-Eur,<insert version>,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,100 nodes,True,3517891,1681180,NaN,NaN,NaN,NaN
pypsa-eur-elec-dfp-50-168h,pypsa-eur-elec-dfp,50-168h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,168 hours,50 nodes,False,134766,64877,NaN,NaN,NaN,NaN
pypsa-eur-elec-dfp-100-12h,pypsa-eur-elec-dfp,100-12h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,100 nodes,True,3517335,1680902,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_vopt-dfp-50-168h,pypsa-eur-elec-trex_vopt-dfp,50-168h,PyPSA,PyPSA-Eur,<insert version>,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,168 hours,50 nodes,False,135086,65037,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_vopt-dfp-100-12h,pypsa-eur-elec-trex_vopt-dfp,100-12h,PyPSA,PyPSA-Eur,<insert version>,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,100 nodes,True,3517891,1681180,NaN,NaN,NaN,NaN
pypsa-eur-elec-trex_copt-dfp-50-168h,pypsa-eur-elec-trex_copt-dfp,50-168h,PyPSA,PyPSA-Eur,<insert version>,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,168 hours,50 nodes,False,135086,65037,NaN,NaN,NaN,NaN


In [161]:
# Create benchmark campaign
l_to_run = benchs_to_run.query('Size == "L"')
vm_yamls = allocate_benchmarks(
    l_to_run,
    "Num. variables",
    len(l_to_run),
    machine_type="c4-highmem-16",
    timeout_seconds=24 * 60 * 60,
)
vm_yamls += allocate_benchmarks(
    benchs_to_run.query('Size != "L"'), "Num. variables", 20
)

create_benchmark_campaign("20260102-more-pypsa", "more-pypsa", vm_yamls)

Allocated. Estimated runtime: 621.7h
  VM 00: 1 instances, 621.7h
  VM 01: 1 instances, 621.7h
  VM 02: 1 instances, 621.6h
  VM 03: 1 instances, 497.5h
  VM 04: 1 instances, 497.5h
  VM 05: 1 instances, 497.5h
  VM 06: 1 instances, 467.0h
  VM 07: 1 instances, 467.0h
  VM 08: 1 instances, 467.0h
  VM 09: 1 instances, 467.0h
  VM 10: 1 instances, 466.9h
  VM 11: 1 instances, 357.0h
  VM 12: 1 instances, 357.0h
  VM 13: 1 instances, 357.0h
Allocated. Estimated runtime: 127.5h
  VM 00: 1 instances, 127.5h
  VM 01: 1 instances, 127.5h
  VM 02: 1 instances, 100.1h
  VM 03: 1 instances, 100.1h
  VM 04: 1 instances, 100.1h
  VM 05: 1 instances, 100.1h
  VM 06: 1 instances, 100.1h
  VM 07: 1 instances, 91.4h
  VM 08: 1 instances, 91.4h
  VM 09: 1 instances, 89.3h
  VM 10: 1 instances, 89.3h
  VM 11: 1 instances, 89.2h
  VM 12: 1 instances, 89.2h
  VM 13: 1 instances, 18.7h
  VM 14: 1 instances, 18.1h
  VM 15: 1 instances, 18.1h
  VM 16: 1 instances, 18.1h
  VM 17: 1 instances, 18.1h
  VM 18: 

In [162]:
# Check in-progress results for runtime variability, in case something needs to be killed and re-run

fetch_all_partial_results()

partial_results, partial_variability = load_results(
    ["../results/gcp-results/20260102-more-pypsa/", "../results/partial-results/"]
)

partial_variability.sort_values("std %", ascending=False)

Cleared ../results/partial-results
There are 29 running VMs. Fetching results from: benchmark-instance-more-pypsa-06	us-central1-a benchmark-instance-more-pypsa-03	us-central1-a benchmark-instance-more-pypsa-04	us-central1-a benchmark-instance-more-pypsa-13	us-central1-a benchmark-instance-more-pypsa-14	us-central1-a benchmark-instance-more-pypsa-00	us-central1-a benchmark-instance-more-pypsa-12	us-central1-a benchmark-instance-more-pypsa-09	us-central1-a benchmark-instance-more-pypsa-19	us-central1-a benchmark-instance-more-pypsa-18	us-central1-a benchmark-instance-more-pypsa-15	us-central1-a benchmark-instance-more-pypsa-08	us-central1-a benchmark-instance-more-pypsa-10	us-central1-a benchmark-instance-more-pypsa-01	us-central1-a benchmark-instance-more-pypsa-16	us-central1-a benchmark-instance-more-pypsa-17	us-central1-a benchmark-instance-more-pypsa-02	us-central1-a benchmark-instance-more-pypsa-05	us-central1-a benchmark-instance-more-pypsa-11	us-central1-a benchmark-instance-more

/Users/sid/code/solver-benchmark/runner/utils.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)


Runtime (s)  \
                                                                         count   
Hostname                         Run ID              VM Zone                     
benchmark-instance-more-pypsa-18 20260102-more-pypsa us-central1-a           9   
benchmark-instance-more-pypsa-23 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-25 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-29 20260102-more-pypsa us-central1-a           5   
benchmark-instance-more-pypsa-20 20260102-more-pypsa us-central1-a           8   
benchmark-instance-more-pypsa-26 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-19 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-22 20260102-more-pypsa us-central1-a           6   
benchmark-instance-more-pypsa-31 20260102-more-pypsa us-central1-a           5   
benchmark-instance-more-pypsa-16 20260102-more-pypsa us-central1-a           8   
benchmark-instance-new-pypsa-16  20251227-new-pypsa  us-central1-a           5   
benchmark-instance-more-pypsa-15 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-21 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-14 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-17 20260102-more-pypsa us-central1-a           8   
benchmark-instance-new-pypsa-15  20251227-new-pypsa  us-central1-a           5   
benchmark-instance-more-pypsa-28 20260102-more-pypsa us-central1-a           5   
benchmark-instance-more-pypsa-33 20260102-more-pypsa us-central1-a           5   
benchmark-instance-more-pypsa-24 20260102-more-pypsa us-central1-a           7   
benchmark-instance-more-pypsa-32 20260102-more-pypsa us-central1-a           5   
benchmark-instance-more-pypsa-27 20260102-more-pypsa us-central1-a           5   
benchmark-instance-more-pypsa-30 20260102-more-pypsa us-central1-a           5   

                                                                                \
                                                                           min   
Hostname                         Run ID              VM Zone                     
benchmark-instance-more-pypsa-18 20260102-more-pypsa us-central1-a  220.918433   
benchmark-instance-more-pypsa-23 20260102-more-pypsa us-central1-a  219.305153   
benchmark-instance-more-pypsa-25 20260102-more-pypsa us-central1-a  211.420679   
benchmark-instance-more-pypsa-29 20260102-more-pypsa us-central1-a  216.558565   
benchmark-instance-more-pypsa-20 20260102-more-pypsa us-central1-a  216.079617   
benchmark-instance-more-pypsa-26 20260102-more-pypsa us-central1-a  211.553283   
benchmark-instance-more-pypsa-19 20260102-more-pypsa us-central1-a  222.907454   
benchmark-instance-more-pypsa-22 20260102-more-pypsa us-central1-a  225.217321   
benchmark-instance-more-pypsa-31 20260102-more-pypsa us-central1-a  188.946978   
benchmark-instance-more-pypsa-16 20260102-more-pypsa us-central1-a  228.427064   
benchmark-instance-new-pypsa-16  20251227-new-pypsa  us-central1-a  213.227115   
benchmark-instance-more-pypsa-15 20260102-more-pypsa us-central1-a  218.481992   
benchmark-instance-more-pypsa-21 20260102-more-pypsa us-central1-a  217.180763   
benchmark-instance-more-pypsa-14 20260102-more-pypsa us-central1-a  215.036371   
benchmark-instance-more-pypsa-17 20260102-more-pypsa us-central1-a  220.112372   
benchmark-instance-new-pypsa-15  20251227-new-pypsa  us-central1-a  215.045358   
benchmark-instance-more-pypsa-28 20260102-more-pypsa us-central1-a  220.659038   
benchmark-instance-more-pypsa-33 20260102-more-pypsa us-central1-a  217.988310   
benchmark-instance-more-pypsa-24 20260102-more-pypsa us-central1-a  220.216075   
benchmark-instance-more-pypsa-32 20260102-more-pypsa us-central1-a  235.377476   
benchmark-instance-more-pypsa-27 20260102-more-pypsa us-central1-a  216.395326   
benchmark-instance-more-pypsa-30 20260102-more-p

### 20260217 Re-run high variance & errored benchmarks

In [27]:
to_run = {
    # These had runtime variance > 6%
    "pypsa-eur-elec-trex_vopt-dfp-100-12h",
    "pypsa-eur-sec-50-168h",
    "pypsa-de-sec-20-12h",
    "SWITCH-3-zone-tiny-3-6ts",
    "Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizon12_Day29-73-1h",
    "Sienna_modified_RTS_GMLC_DA_sys_NetTransport_Horizon12_Day314-73-1h",
    "pglib_opf_case2848-2848-NA",
    "times-etimeseu-europe-elec+heat-co2-single_stage-29-64ts",
    "pypsa-de-elec-trex_vopt-20-1h",
    # This one errored and needs to be re-run
    "Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizon48_Day314-73-1h",
}
benchmarks_df = load_benchmark_metadata()
benchs_to_run = benchmarks_df[benchmarks_df.index.isin(to_run)].copy()
benchs_to_run

,Benchmark,Instance,Modelling framework,Model name,Version,Contributor(s)/Source,License,Problem class,Application,Sectoral focus,...,URL,Temporal resolution,Spatial resolution,Realistic,Num. constraints,Num. variables,Skip because,Num. continuous variables,Num. integer variables,Notes
pglib_opf_case2848-2848-NA,pglib_opf_case2848,2848-NA,PowerModels,PowerModels,None,https://github.com/jump-dev/open-energy-modeli...,"Model code licensed MIT, benchmark instances n...",MILP,Steady-state Optimal Power Flow,Power-only,...,https://raw.githubusercontent.com/jump-dev/ope...,NA,2848 nodes,False,25505,10911,NaN,7135.0,3776.0,NaN
Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizon12_Day29-73-1h,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,73-1h,Sienna,Sienna,None,https://github.com/jump-dev/open-energy-modeli...,"Model code licensed MIT, benchmark instances n...",MILP,Infrastructure,Power-only,...,https://raw.githubusercontent.com/jump-dev/ope...,12 hours,73 nodes,False,10368,8820,NaN,6876.0,1944.0,NaN
Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizon48_Day314-73-1h,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,73-1h,Sienna,Sienna,None,https://github.com/jump-dev/open-energy-modeli...,"Model code licensed MIT, benchmark instances n...",MILP,Infrastructure,Power-only,...,https://raw.githubusercontent.com/jump-dev/ope...,48 hours,73 nodes,False,41472,35280,NaN,27504.0,7776.0,NaN
Sienna_modified_RTS_GMLC_DA_sys_NetTransport_Horizon12_Day314-73-1h,Sienna_modified_RTS_GMLC_DA_sys_NetTransport_H...,73-1h,Sienna,Sienna,None,https://github.com/jump-dev/open-energy-modeli...,"Model code licensed MIT, benchmark instances n...",MILP,Infrastructure,Power-only,...,https://raw.githubusercontent.com/jump-dev/ope...,12 hours,73 nodes,False,9792,8820,NaN,6876.0,1944.0,NaN
pypsa-eur-elec-trex_vopt-dfp-100-12h,pypsa-eur-elec-trex_vopt-dfp,100-12h,PyPSA,PyPSA-Eur,<insert version>,"Daniele Lerede, Open Energy Transition",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,12 hours,100 nodes,True,3517891,1681180,NaN,NaN,NaN,NaN
pypsa-eur-sec-50-168h,pypsa-eur-sec,50-168h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Sector-coupled,...,https://storage.googleapis.com/solver-benchmar...,24 hours,50 nodes,False,677039,321282,NaN,NaN,NaN,NaN
pypsa-de-elec-trex_vopt-20-1h,pypsa-de-elec-trex_vopt,20-1h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,1 hour,20 nodes,True,8208517,3854670,NaN,NaN,NaN,NaN
pypsa-de-sec-20-12h,pypsa-de-sec,20-12h,PyPSA,PyPSA-Eur,None,"Daniele Lerede, Open Energy Transition; Fabriz...",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Sector-coupled,...,https://storage.googleapis.com/solver-benchmar...,12 hours,20 nodes,False,3797984,1791010,NaN,NaN,NaN,NaN
SWITCH-3-zone-tiny-3-6ts,SWITCH-3-zone-tiny,3-6ts,SWITCH,3_zone_tiny,None,"Vamsi Priya Goli, Open Energy Transition; http...",CC BY 4.0,MILP,Infrastructure & Capacity Expansion,Power-only,...,https://storage.googleapis.com/solver-benchmar...,"6 time slices (first investment period), 1 tim...",3 nodes,False,795,582,NaN,580.0,2.0,NaN
times-etimeseu-europe-elec+heat-co2-single_stage-29-64ts,times-etimeseu-europe-elec+heat-co2-single_stage,29-64ts,TIMES,eTIMES-EU,None,"Victor Guillot, École des Mines de Paris",CC BY 4.0,LP,Infrastructure & Capacity Expansion,Sector-coupled,...,https://storage.googleapis.com/solver-benchmar...,64 time slices,29 regions,True,456663,452914,NaN,NaN,NaN,NaN


In [29]:
# Create benchmark campaign
l_to_run = benchs_to_run.query('Size == "L"')
vm_yamls = allocate_benchmarks(
    l_to_run,
    "Num. variables",
    len(l_to_run),
    machine_type="c4-highmem-16",
    timeout_seconds=24 * 60 * 60,
    years=[2024, 2025],  # latest solvers only, so skip creating older conda envs
)
vm_yamls += allocate_benchmarks(benchs_to_run.query('Size != "L"'), "Num. variables", 5)

create_benchmark_campaign("20260217-rerun-4", "rerun-4", vm_yamls)

Allocated. Estimated runtime: 1070.7h
  VM 00: 1 instances, 1070.7h
  VM 01: 1 instances, 497.5h
  VM 02: 1 instances, 467.0h
Allocated. Estimated runtime: 125.8h
  VM 00: 1 instances, 125.8h
  VM 01: 1 instances, 89.2h
  VM 02: 1 instances, 9.8h
  VM 03: 2 instances, 3.2h
  VM 04: 2 instances, 4.9h
Created directory and files in ../infrastructure/benchmarks/20260217-rerun-4
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20260217-rerun-4/run.tfvars -state=states/20260217-rerun-4.tfstate


### 20260219 Re-run SWITCH-3-zone-tiny
In the previous run, the URL was incorrect, so I manually copied the yaml file and corrected it. Ran with the command:
```
tofu apply -var-file benchmarks/20260219-rerun-4/run.tfvars -state=states/20260219-rerun-4.tfstate
```

### 20260223 Re-run pglib_opf_case2848
Forgot to re-run this, even though it had variation > 6%.

In [16]:
to_run = {
    "pglib_opf_case2848-2848-NA",
}
benchmarks_df = load_benchmark_metadata()
benchs_to_run = benchmarks_df[benchmarks_df.index.isin(to_run)].copy()
vm_yamls = allocate_benchmarks(benchs_to_run, "Num. variables", 1)
create_benchmark_campaign("20260223-rerun-4", "rerun-4", vm_yamls)

Allocated. Estimated runtime: 3.0h
  VM 00: 1 instances, 3.0h
Created directory and files in ../infrastructure/benchmarks/20260223-rerun-4
Run this campaign from the infrastructure/ directory using the command:
tofu apply -var-file benchmarks/20260223-rerun-4/run.tfvars -state=states/20260223-rerun-4.tfstate


## Run and monitor runs

In [3]:
check_uptimes()

There are 1 running instances
benchmark-instance-00-campaign: 06:30:23 up 2 days, 15:32,  1 user,  load average: 1.09, 1.04, 1.08

0 potentially hung instances:



In [4]:
fetch_all_partial_results()

Cleared ../results/partial-results
There are 1 running VMs. Fetching results from: benchmark-instance-00-campaign	us-central1-a Done.


## Validate results

Results were downloaded by running the following command on the root directory of this repository:
```
gsutil -m rsync -r gs://solver-benchmarks/logs ./runner/logs/ && gsutil -m rsync -r gs://solver-benchmarks-restricted/logs ./runner/logs/  && gsutil -m rsync -r gs://solver-benchmarks/results ./results/gcp-results/
```
Then, the rest of the cells in this section were run to clean, check, and export results to the website.

In [16]:
results, variability = load_results(
    [
        "../results/gcp-results/20251212-run-Ls/",
        "../results/gcp-results/20251214-rerun-1/",
        "../results/gcp-results/20251215-run-S-M/",
        "../results/gcp-results/20251216-rerun-2/",
        # "../results/gcp-results/20251219-pypsa-sizes/", -- only ran highs variants
        "../results/gcp-results/20251222-leftovers/",
        "../results/gcp-results/20251227-new-pypsa/",
        "../results/gcp-results/20251228-rerun-3/",
        "../results/gcp-results/20260102-more-pypsa/",
        "../results/gcp-results/20260217-rerun-4/",
        "../results/gcp-results/20260219-rerun-4/",
        "../results/gcp-results/20260223-rerun-4/",
        # "../results/partial-results/",
    ]
)
benchmarks_df = load_benchmark_metadata()

/Users/sid/code/solver-benchmark/notebooks/../runner/utils.py:160: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([pd.read_csv(p) for p in csv_files]).reset_index(drop=True)


Found 3378 records, 220 benchmark instances


### Fix bench-size-size bug
Unfortunately, `utils.allocate_benchmarks` had a bug where it would append the size name to the bench name in the `Benchmark` column of the results for some of the campaigns above, so this needs to be fixed before the results can be passed on to the website, otherwise it will break the link between benchmark metadata and results.

In [17]:
# Remove the size name from the Benchmark column
results["Benchmark1"] = results.apply(
    lambda r: r["Benchmark"][: -len(r["Size"]) - 1]
    if r["Benchmark"].endswith(r["Size"])
    else r["Benchmark"],
    axis=1,
)
results.query("Benchmark != Benchmark1").shape

(2074, 23)

In [4]:
# Rename the log and solution files accordingly

# First, run these in a terminal to obtain the list of filenames in the bucket:
"""
mkdir -p /tmp/bench-size-size; cd /tmp/bench-size-size
gsutil ls 'gs://solver-benchmarks/logs/**' > logs_solns.txt
gsutil ls 'gs://solver-benchmarks-restricted/logs/**' >> logs_solns.txt 
gsutil ls 'gs://solver-benchmarks/solutions/**' >> logs_solns.txt 
"""
existing_files = set(open("/tmp/bench-size-size/logs_solns.txt").readlines())

cmds = []
for _, r in results.iterrows():
    if r["Benchmark"] != r["Benchmark1"]:
        # Rename the solution
        old_path = f"gs://solver-benchmarks/solutions/{r['Run ID']}/{r['bench-size']}-{r['solver-version']}.sol.gz"
        new_path = f"gs://solver-benchmarks/solutions/{r['Run ID']}/{r['Benchmark1']}-{r['Size']}-{r['solver-version']}.sol.gz"
        if old_path in existing_files:
            cmds.append(f"gsutil mv {old_path} {new_path}")
        # Rename the log
        if r["Solver"] == "gurobi":
            bucket = "solver-benchmarks-restricted"
        else:
            bucket = "solver-benchmarks"
        old_path = f"gs://{bucket}/logs/{r['Run ID']}/{r['bench-size']}-{r['solver-version']}.log.gz"
        new_path = f"gs://{bucket}/logs/{r['Run ID']}/{r['Benchmark1']}-{r['Size']}-{r['solver-version']}.log.gz"
        if old_path in existing_files:
            cmds.append(f"gsutil mv {old_path} {new_path}")
with open("/tmp/bench-size-size/mv_logs_solns.sh", "w") as f:
    f.write("\n".join(cmds))

In [18]:
# Overwrite Benchmark column before writing results to website
results["Benchmark"] = results["Benchmark1"]
results["bench-size"] = results["Benchmark"] + "-" + results["Size"]
print(
    f"After correction: {len(results)} records, {len(results['bench-size'].unique())} benchmark instances"
)

After correction: 3378 records, 214 benchmark instances


### Drop results superceeded by newer runs

In [19]:
# Find the Run IDs and Hostnames for each bench-size instance and drop all but the latest
# NOTE: assumes all Run IDs begin with YYYYMMDD-
runs_grouped = results.groupby("bench-size")[["Run ID", "Hostname"]]
to_drop = set()
for bench_size, group in runs_grouped:
    unique_runs = group[["Run ID", "Hostname"]].drop_duplicates()
    if len(unique_runs) > 1:
        sorted_runs = sorted(unique_runs.itertuples(index=False))
        to_drop.update([(*run, bench_size) for run in sorted_runs[:-1]])

print("Dropping results from these runs:", sorted(to_drop))
keys = pd.MultiIndex.from_frame(results[["Run ID", "Hostname", "bench-size"]])
results = results.loc[~keys.isin(to_drop)].copy()
print(
    f"After dropping: {len(results)} records, {len(results['bench-size'].unique())} benchmark instances"
)

Dropping results from these runs: [('20251212-run-Ls', 'benchmark-instance-l-00', 'genx-elec_trex-15-168h'), ('20251212-run-Ls', 'benchmark-instance-l-01', 'genx-elec_trex_co2-15-168h'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'DCOPF-Carolinas_1W-997-1h'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'DCOPF-Carolinas_uc_1W-997-1h'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'SWITCH-3-zone-tiny-3-6ts'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizon12_Day29-73-1h'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'Sienna_modified_RTS_GMLC_DA_sys_NetTransport_Horizon12_Day314-73-1h'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'Sienna_modified_RTS_GMLC_DA_sys_NetTransport_Horizon48_Day314-73-1h'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'pglib_opf_case2848-2848-NA'), ('20251214-rerun-1', 'benchmark-instance-s-m-06', 'times-etimeseu-europe-elec+heat-co2-single_stage-29-64ts'), ('20251215-run-S-M'

### Validate runtime variability

In [20]:
# Calculate the runtime variability for each benchmark instance

runtimes = results.query('Status == "ok"')
runtimes = runtimes.groupby(["bench-size"])["Runtime (s)"].agg(list)

runtime_stats = pd.DataFrame(
    {
        "runtimes": runtimes,
        "mean_runtime": runtimes.apply(lambda x: sum(x) / len(x)),
    }
)
runtime_stats["VM"] = runtime_stats.index.map(
    lambda bs: (
        results[results["bench-size"] == bs]["Hostname"].iloc[0],
        results[results["bench-size"] == bs]["Run ID"].iloc[0],
        results[results["bench-size"] == bs]["VM Zone"].iloc[0],
    )
    if any(results["bench-size"] == bs)
    else None
)
runtime_stats["observed_var"] = runtime_stats.apply(
    lambda r: variability.loc[r["VM"], ("std %", "")], axis=1
)
runtime_stats.query("observed_var > 6").sort_values("observed_var", ascending=False)

,runtimes,mean_runtime,VM,observed_var
bench-size,,,,
pypsa-de-elec-trex_vopt-50-3h,[703.5494537499908],703.549454,"(benchmark-instance-new-pypsa-09, 20251227-new...",9.414971
pypsa-de-elec-trex_vopt-dfp-50-3h,[623.0094539709971],623.009454,"(benchmark-instance-new-pypsa-08, 20251227-new...",8.642247
DCOPF-Carolinas_1W-997-1h,"[5.574290518998168, 11.118886923999526, 89.174...",32.812983,"(benchmark-instance-s-m-06, 20251216-rerun-2, ...",6.367322
DCOPF-Carolinas_uc_1W-997-1h,"[454.0450894739988, 3308.543003070001, 388.949...",1664.881474,"(benchmark-instance-s-m-06, 20251216-rerun-2, ...",6.367322


In [21]:
# Drop some benchmark instances that have high variation and aren't interesting enough to re-run

to_drop = {
    "pypsa-de-elec-trex_vopt-50-3h",
    "pypsa-de-elec-trex_vopt-dfp-50-3h",
    "DCOPF-Carolinas_1W-997-1h",
    "DCOPF-Carolinas_uc_1W-997-1h",
}

results = results[~results["bench-size"].isin(to_drop)]

to_rerun = runtime_stats.query("observed_var > 6").sort_values(
    "observed_var", ascending=False
)
to_rerun = to_rerun[~to_rerun.index.isin(to_drop)]
print(len(to_rerun["VM"].unique()))
to_rerun

0


,runtimes,mean_runtime,VM,observed_var
bench-size,,,,


Thus, all benchmarks in results have been executed on VMs where the observed variation in runtime was <= 6%.

### Check results are complete

In [22]:
from collections import Counter

seen_solvers = Counter()
for bench_size, group in results.groupby("bench-size"):
    solvers_present = sorted(group["solver-version"])
    seen_solvers[str(solvers_present)] += 1
    if len(solvers_present) not in {4, 6, 15, 17}:
        print("Odd combination of solvers run on", bench_size, solvers_present)

seen_solvers

Counter({"['cbc-2.10.11', 'cbc-2.10.12', 'glpk-5.0', 'gurobi-10.0.0', 'gurobi-11.0.0', 'gurobi-12.0.0', 'gurobi-13.0.0', 'highs-1.12.0', 'highs-1.5.0.dev0', 'highs-1.6.0.dev0', 'highs-1.9.0', 'scip-10.0.0', 'scip-8.0.3', 'scip-8.1.0', 'scip-9.2.0']": 78,
         "['cbc-2.10.11', 'cbc-2.10.12', 'glpk-5.0', 'gurobi-10.0.0', 'gurobi-11.0.0', 'gurobi-12.0.0', 'gurobi-13.0.0', 'highs-1.12.0', 'highs-1.5.0.dev0', 'highs-1.6.0.dev0', 'highs-1.9.0', 'highs-hipo-1.12.0-hipo', 'highs-ipm-1.12.0-hipo', 'scip-10.0.0', 'scip-8.0.3', 'scip-8.1.0', 'scip-9.2.0']": 73,
         "['cbc-2.10.12', 'gurobi-13.0.0', 'highs-1.12.0', 'highs-hipo-1.12.0-hipo', 'highs-ipm-1.12.0-hipo', 'scip-10.0.0']": 57,
         "['cbc-2.10.12', 'gurobi-13.0.0', 'highs-1.12.0', 'scip-10.0.0']": 2})

In [ ]:
# For v2, we ran all years' solvers on S & M, and only latest ones on L. We also ran hipo and ipm variants of highs on LPs.
# TODO check the above is true in the results we have

In [23]:
# Check that there are no benchmarks in results/metadata.yaml that don't have results yet
benchs_available = benchmarks_df[benchmarks_df["Skip because"].isna()].index
benchs_run_so_far = set(results["bench-size"].unique())
print("Benchmarks without results:", sorted(set(benchs_available) - benchs_run_so_far))
# pypsa-de/eur are still running. Others are ones we've chosen to skip

Benchmarks without results: []


In [24]:
# Check that each instance only has results from 1 VM
found_error = False
for bench_size, group in results.groupby("bench-size")[["Run ID", "Hostname"]]:
    unique_hosts = group[["Run ID", "Hostname"]].drop_duplicates()
    if len(unique_hosts) > 1:
        print(f"ERROR: {bench_size} has results from > 1 VM: {unique_hosts}")
        found_error = True
if not found_error:
    print("All benchmarks have results from only one VM.")

All benchmarks have results from only one VM.


### Check for solver errors and warnings

In [11]:
# Export error rows to spreadsheet
error_results = results.query('Status != "ok" and Status != "TO" and Status != "OOM"')[
    [
        "bench-size",
        "solver-version",
        "Run ID",
        "Hostname",
        "Status",
        "Termination Condition",
    ]
]
error_results.to_csv("~/Downloads/v2-errors.csv", index=False)

In [25]:
# Print any potential errors in latest solvers:
latest_solvers = {
    "cbc-2.10.12",
    "gurobi-13.0.0",
    "highs-hipo-1.12.0-hipo",
    "highs-ipm-1.12.0-hipo",
    "highs-1.12.0",
    "scip-10.0.0",
}
latest_results = results[results["solver-version"].isin(latest_solvers)]
latest_results.query('Status != "ok" and Status != "TO" and Status != "OOM"')[
    ["bench-size", "solver-version", "Status", "Timestamp", "Run ID", "Hostname"]
]

,bench-size,solver-version,Status,Timestamp,Run ID,Hostname
2,TIMES-GEO-global-netzero-31-20ts,highs-hipo-1.12.0-hipo,warning,2025-12-13 04:01:38.198589,20251212-run-Ls,benchmark-instance-l-19
4,TIMES-GEO-global-netzero-31-20ts,highs-ipm-1.12.0-hipo,warning,2025-12-13 22:03:17.152733,20251212-run-Ls,benchmark-instance-l-19
38,times-ireland-noco2-counties-26-1ts,highs-ipm-1.12.0-hipo,warning,2025-12-12 19:45:46.110622,20251212-run-Ls,benchmark-instance-l-23
43,times-ireland-noco2-counties-26-1ts,cbc-2.10.12,ER,2025-12-13 21:29:04.002430,20251212-run-Ls,benchmark-instance-l-23
66,pypsa-de-sec-trex_copt-50-1h,cbc-2.10.12,ER,2025-12-14 21:00:43.095362,20251212-run-Ls,benchmark-instance-l-11
94,times-etimeseu-europe-elec+heat-co2-multi_stag...,cbc-2.10.12,ER,2025-12-16 20:06:05.043815,20251212-run-Ls,benchmark-instance-l-21
124,genx-elec_trex_uc-15-24h,scip-10.0.0,ER,2025-12-13 19:44:38.068006,20251212-run-Ls,benchmark-instance-l-02
125,genx-elec_trex_uc-15-24h,cbc-2.10.12,ER,2025-12-13 19:45:13.123907,20251212-run-Ls,benchmark-instance-l-02
147,TIMES-GEO-global-base-31-20ts,highs-ipm-1.12.0-hipo,warning,2025-12-13 20:06:42.709401,20251212-run-Ls,benchmark-instance-l-18
185,times-ireland-noco2-40-1ts,cbc-2.10.12,ER,2025-12-16 20:15:48.795506,20251212-run-Ls,benchmark-instance-l-22


These errors have been investigated and dismissed:
- GLPK also errrors in v1 on: temoa-utopia, times-etimeseu-france-elec+heat-co2-single_stage, Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizon24_Day332, 
- pglib_opf_case162_ieee_dtc also infeasible in v1

Did IPM run crossover for times-ireland-noco2-counties-26-1ts?

To open issues:
- CBC model not valid: times-etimeseu-europe-elec+heat-single_stage, times-etimeseu-france-elec+heat-single_stage, all etimeseu / times-nz, times-ireland-noco2-counties-26-1ts
- CBC read error: genx-elec_trex_uc-15-24h
- SCIP read error: genx-10_IEEE_9_bus_DC_OPF-no_uc (it ran successfully in v1, perhaps error with pyscipopt?), genx-elec_trex_uc-15-24h
- SCIP Error <-6> in function call: OEMOF-v4/v3


TODO investigate:
- runner/logs/20251212-run-Ls/pypsa-de-sec-trex_vopt-50-1h-cbc-2.10.12.log.gz (re-running on ZIB produces no output? It's not an OOM..)

In [21]:
# TODO open linopy issue? Pehaps these shouldn't be counted as a successful solve:
results.query(
    'Status == "ok" and `Termination Condition` != "optimal" and `Termination Condition` != "Optimal"'
)

,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,...,Timeout,Hostname,Run ID,Timestamp,VM Instance Type,VM Zone,Solver benchmark version,bench-size,solver-version,Benchmark1
6,TIMES-GEO-global-netzero,31-20ts,highs,1.12.0,2025.0,ok,unknown,62869.744979,17229.104,0.000000e+00,...,86400.0,benchmark-instance-l-19,20251212-run-Ls,2025-12-14 17:34:17.922231,c4-highmem-16,europe-north2-b,90fca5f,TIMES-GEO-global-netzero-31-20ts,highs-1.12.0,TIMES-GEO-global-netzero
775,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,73-1h,glpk,5.0,2020.0,ok,unknown,3359.225640,438.480,2.388675e+05,...,3600.0,benchmark-instance-s-m-00,20251215-run-S-M,2025-12-15 20:00:03.520866,c4-standard-2,europe-west2-a,48cfc10,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,glpk-5.0,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...
906,Sienna_modified_RTS_GMLC_DA_sys_NetDC_Horizon1...,73-1h,glpk,5.0,2020.0,ok,unknown,341.209363,135.808,2.383709e+05,...,3600.0,benchmark-instance-s-m-07,20251215-run-S-M,2025-12-15 20:46:07.125279,c4-standard-2,europe-west8-c,48cfc10,Sienna_modified_RTS_GMLC_DA_sys_NetDC_Horizon1...,glpk-5.0,Sienna_modified_RTS_GMLC_DA_sys_NetDC_Horizon1...
907,Sienna_modified_RTS_GMLC_DA_sys_NetCopperPlate...,73-1h,glpk,5.0,2020.0,ok,unknown,168.081790,130.688,2.220033e+05,...,3600.0,benchmark-instance-s-m-07,20251215-run-S-M,2025-12-15 20:51:48.749080,c4-standard-2,europe-west8-c,48cfc10,Sienna_modified_RTS_GMLC_DA_sys_NetCopperPlate...,glpk-5.0,Sienna_modified_RTS_GMLC_DA_sys_NetCopperPlate...
1037,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,73-1h,glpk,5.0,2020.0,ok,unknown,1862.678395,606.088,2.899119e+05,...,3600.0,benchmark-instance-s-m-13,20251215-run-S-M,2025-12-15 20:42:07.195777,c4-standard-2,us-west3-a,48cfc10,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,glpk-5.0,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...
1039,Sienna_modified_RTS_GMLC_DA_sys_NetTransport_H...,73-1h,glpk,5.0,2020.0,ok,unknown,245.493613,133.256,2.219681e+05,...,3600.0,benchmark-instance-s-m-13,20251215-run-S-M,2025-12-15 21:16:14.963400,c4-standard-2,us-west3-a,48cfc10,Sienna_modified_RTS_GMLC_DA_sys_NetTransport_H...,glpk-5.0,Sienna_modified_RTS_GMLC_DA_sys_NetTransport_H...
1040,genx-7_three_zones_w_colocated_VRE_storage,3-24h,glpk,5.0,2020.0,ok,unknown,0.664741,128.228,4.433344e+05,...,3600.0,benchmark-instance-s-m-13,20251215-run-S-M,2025-12-15 21:20:20.934400,c4-standard-2,us-west3-a,48cfc10,genx-7_three_zones_w_colocated_VRE_storage-3-24h,glpk-5.0,genx-7_three_zones_w_colocated_VRE_storage
1681,tulipa-1_EU_investment_simple,28-4.3h,glpk,5.0,2020.0,ok,unknown,3026.944333,656.596,3.094031e+09,...,3600.0,benchmark-instance-s-m-09,20251215-run-S-M,2025-12-15 19:39:39.281020,c4-standard-2,europe-west3-a,48cfc10,tulipa-1_EU_investment_simple-28-4.3h,glpk-5.0,tulipa-1_EU_investment_simple
1683,pypsa-power+ely-mod,1-1h,glpk,5.0,2020.0,ok,unknown,91.131943,229.952,3.171513e+10,...,3600.0,benchmark-instance-s-m-09,20251215-run-S-M,2025-12-15 20:30:07.375832,c4-standard-2,europe-west3-a,48cfc10,pypsa-power+ely-mod-1-1h,glpk-5.0,pypsa-power+ely-mod
1684,tulipa-1_EU_investment_simple,28-24h,glpk,5.0,2020.0,ok,unknown,2.224164,129.848,2.133202e+08,...,3600.0,benchmark-instance-s-m-09,20251215-run-S-M,2025-12-15 20:31:39.029294,c4-standard-2,europe-west3-a,48cfc10,tulipa-1_EU_investment_simple-28-24h,glpk-5.0,tulipa-1_EU_investment_simple


#### Hipo/ipm unknown statuses

In [ ]:
for _, r in results.query(
    'Status != "ok" and Status != "TO" and Status != "OOM" and Solver == "highs-hipo"'
).iterrows():
    print("\n", r["bench-size"], "\n")
    !zless ../runner/logs/20251209-test-S-Ms/{r['bench-size']}-{r['Solver']}-{r['Solver Version']}.log.gz | tail -n 20

In [22]:
# results.query('`bench-size` == "times-etimeseu-europe-elec+heat-single_stage-29-64ts"')
# results.query('`bench-size` == "DCOPF-Carolinas_2M-997-1h"')
results.query('Solver == "highs-hipo" and Status == "warning"')

,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,...,Timeout,Hostname,Run ID,Timestamp,VM Instance Type,VM Zone,Solver benchmark version,bench-size,solver-version,Benchmark1
2,TIMES-GEO-global-netzero,31-20ts,highs-hipo,1.12.0-hipo,2025.0,warning,Not Set,64715.836039,14280.740,NaN,...,86400.0,benchmark-instance-l-19,20251212-run-Ls,2025-12-13 04:01:38.198589,c4-highmem-16,europe-north2-b,90fca5f,TIMES-GEO-global-netzero-31-20ts,highs-hipo-1.12.0-hipo,TIMES-GEO-global-netzero
436,times-nz-kea,2-24ts,highs-hipo,1.12.0-hipo,2025.0,warning,Unknown,459.096089,1402.832,1.213210e+06,...,3600.0,benchmark-instance-s-m-05,20251215-run-S-M,2025-12-17 07:35:13.209719,c4-standard-2,europe-west9-c,48cfc10,times-nz-kea-2-24ts,highs-hipo-1.12.0-hipo,times-nz-kea
754,times-etimeseu-europe-elec+heat-single_stage,29-64ts,highs-hipo,1.12.0-hipo,2025.0,warning,Unknown,43.075509,514.256,4.304514e+05,...,3600.0,benchmark-instance-s-m-11,20251215-run-S-M,2025-12-17 09:47:09.404641,c4-standard-2,europe-west3-c,48cfc10,times-etimeseu-europe-elec+heat-single_stage-2...,highs-hipo-1.12.0-hipo,times-etimeseu-europe-elec+heat-single_stage
1232,temoa-utopia,1-6ts,highs-hipo,1.12.0-hipo,2025.0,warning,Unknown,0.099804,163.892,3.653563e+04,...,3600.0,benchmark-instance-s-m-13,20251215-run-S-M,2025-12-17 00:37:35.585736,c4-standard-2,us-west3-a,48cfc10,temoa-utopia-1-6ts,highs-hipo-1.12.0-hipo,temoa-utopia
1946,times-nz-tui,2-24ts,highs-hipo,1.12.0-hipo,2025.0,warning,Unknown,388.395358,1437.300,1.402087e+06,...,3600.0,benchmark-instance-s-m-03,20251215-run-S-M,2025-12-17 06:04:40.443791,c4-standard-2,europe-west9-a,48cfc10,times-nz-tui-2-24ts,highs-hipo-1.12.0-hipo,times-nz-tui
4678,times-etimeseu-europe-elec+heat-co2-single_stage,29-64ts,highs-hipo,1.12.0-hipo,2025.0,warning,Unknown,73.417504,584.500,4.320456e+05,...,3600.0,benchmark-instance-rerun-4-03,20260217-rerun-4,2026-02-17 10:23:53.963088,c4-standard-2,us-central1-a,0a76d2f,times-etimeseu-europe-elec+heat-co2-single_sta...,highs-hipo-1.12.0-hipo,times-etimeseu-europe-elec+heat-co2-single_stage


### Validate objective values, MIP metrics

In [26]:
# Check that all solvers report similar objective values


# TODO glpk is a bit off on this benchmark. Create issue!
# results[(results['Benchmark'] == 'tulipa-1_EU_investment_simple') & (results['Size'] == '28-24h')]
# obj_dev = results[(~results['Objective Value'].isna())].groupby(['Benchmark', 'Size']).agg({'Objective Value': ['min', 'max', 'std']})

# TODO only do this for ok/optimal results?
obj_dev = (
    results[(~results["Objective Value"].isna()) & (results["Solver"] != "glpk")]
    .groupby(["Benchmark", "Size"])
    .agg({"Objective Value": ["min", "max", "std"]})
)

obj_dev["std_ratio"] = (
    obj_dev[("Objective Value", "std")] / obj_dev[("Objective Value", "min")]
)
obj_dev[obj_dev["std_ratio"] > 1e-4].sort_values(by="std_ratio", ascending=False)

Objective Value                \
                                                    min           max   
Benchmark                       Size                                    
TIMES-GEO-global-netzero        31-20ts    0.000000e+00  2.173686e+08   
genx-10_IEEE_9_bus_DC_OPF-no_uc 9-1h       1.149079e+06  1.150959e+06   

                                                      std_ratio  
                                                  std            
Benchmark                       Size                             
TIMES-GEO-global-netzero        31-20ts  1.537028e+08       inf  
genx-10_IEEE_9_bus_DC_OPF-no_uc 9-1h     9.817980e+02  0.000854

**TODO** is there a way to automatically list outliers in terms of objective value? E.g. for ('DCOPF-Carolinas_2M', '997-1h') the only outlier is hipo.

In [27]:
results.query('`bench-size` == "TIMES-GEO-global-netzero-31-20ts"')[
    [
        "bench-size",
        "solver-version",
        "Status",
        "Termination Condition",
        "Objective Value",
    ]
]

,bench-size,solver-version,Status,Termination Condition,Objective Value
0,TIMES-GEO-global-netzero-31-20ts,gurobi-13.0.0,ok,optimal,2.173686e+08
2,TIMES-GEO-global-netzero-31-20ts,highs-hipo-1.12.0-hipo,warning,Not Set,NaN
4,TIMES-GEO-global-netzero-31-20ts,highs-ipm-1.12.0-hipo,warning,Not Set,NaN
6,TIMES-GEO-global-netzero-31-20ts,highs-1.12.0,ok,unknown,0.000000e+00
8,TIMES-GEO-global-netzero-31-20ts,scip-10.0.0,TO,Timeout,NaN
10,TIMES-GEO-global-netzero-31-20ts,cbc-2.10.12,TO,Timeout,NaN


In [28]:
# Since highs didn't really solve that successfully, we set Status from ok to ER:
bad_solve_idx = results.query(
    '`bench-size` == "TIMES-GEO-global-netzero-31-20ts" and `solver-version` == "highs-1.12.0"'
).index
results.loc[bad_solve_idx, "Status"] = "ER"

In [29]:
results.query('`bench-size` == "genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h"')[
    [
        "bench-size",
        "solver-version",
        "Status",
        "Termination Condition",
        "Objective Value",
    ]
]
# Here it looks like all the highs solvers report a different solutoin compared to gurobi and cbc. Perhaps we need a solution quality checker here to determine who is right!

,bench-size,solver-version,Status,Termination Condition,Objective Value
1526,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,glpk-5.0,warning,unknown,NaN
1554,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,gurobi-10.0.0,ok,optimal,1.149079e+06
1555,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,highs-1.5.0.dev0,ok,optimal,1.150959e+06
1556,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,scip-8.0.3,ER,NaN,NaN
1594,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,gurobi-11.0.0,ok,optimal,1.149079e+06
1595,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,highs-1.6.0.dev0,ok,optimal,1.150959e+06
1596,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,scip-8.1.0,ER,NaN,NaN
1597,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,cbc-2.10.11,ok,optimal,1.149079e+06
1635,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,gurobi-12.0.0,ok,optimal,1.149079e+06
1636,genx-10_IEEE_9_bus_DC_OPF-no_uc-9-1h,highs-1.9.0,ok,optimal,1.150959e+06


In [30]:
# Check if any results have high duality gap (above 1e-4)
results[results["Duality Gap"] > 1e-4].sort_values(by="Duality Gap", ascending=False)

,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,...,Timeout,Hostname,Run ID,Timestamp,VM Instance Type,VM Zone,Solver benchmark version,bench-size,solver-version,Benchmark1


In [31]:
# Check if any results have high maximum integrality violation
results[results["Max Integrality Violation"] > 1e-5].sort_values(
    by="Max Integrality Violation", ascending=False
)

,Benchmark,Size,Solver,Solver Version,Solver Release Year,Status,Termination Condition,Runtime (s),Memory Usage (MB),Objective Value,...,Timeout,Hostname,Run ID,Timestamp,VM Instance Type,VM Zone,Solver benchmark version,bench-size,solver-version,Benchmark1


### Write results to website

In [32]:
# Rename highs-ipm to highs-ipx
results_web = results.copy()
results_web["Solver"] = results_web["Solver"].map(
    lambda x: "highs-ipx" if x == "highs-ipm" else x
)
results_web["solver-version"] = results_web["solver-version"].map(
    lambda x: "highs-ipx-1.12.0-hipo" if x == "highs-ipm-1.12.0-hipo" else x
)

In [33]:
# Write results to use on website
results_web.to_csv("../results/benchmark_results.csv", index=False)

## Analyze cost of executing v2 benchmarks

In [33]:
# Compute aggregations per bench-size
agg_results = results.groupby("bench-size").agg(
    num_solvers=("solver-version", "nunique"),
    total_runtime=("Runtime (s)", "sum"),
    avg_runtime=("Runtime (s)", "mean"),
)

# Merge with benchmarks_df to get Size
agg_results = agg_results.merge(
    benchmarks_df[["Size"]], left_index=True, right_index=True
)

# Compute CPUh based on Size
agg_results["CPUh"] = agg_results.apply(
    lambda row: row["total_runtime"] * (2 if row["Size"] in ["S", "M"] else 16) / 3600,
    axis=1,
)

In [40]:
from humanize import naturaldelta

total_runtime = results["Runtime (s)"].sum()
print(naturaldelta(total_runtime, months=False))
print(naturaldelta(agg_results["total_runtime"].sum(), months=False))
print(
    naturaldelta(
        (agg_results["avg_runtime"] * agg_results["num_solvers"]).sum(), months=False
    )
)

print(f"Total v2 runtime: {total_runtime:,.0f} ({naturaldelta(total_runtime)})")
print(f"Total CPUh: {agg_results['CPUh'].sum():,.0f}")

268 days
268 days
285 days
Total v2 runtime: 23,196,538 (8 months)
Total CPUh: 91,158


In [41]:
# Cost if we were to have run L benchmarks on all solver versions
c4_std_2_cost, c4_hgm_16_cost = 0.1, 1.05

s_m_runtime = agg_results.query("Size != 'L'")["total_runtime"].sum()
l_runtime = (agg_results.query("Size == 'L'")["total_runtime"]).sum()
total_runtime = s_m_runtime + l_runtime
total_cost = (s_m_runtime * c4_std_2_cost + l_runtime * c4_hgm_16_cost) / 3600
print(
    f"Run v2, total runtime = {total_runtime:,.0f}s ({naturaldelta(total_runtime, months=False)}), cost = ${total_cost:,.0f}"
)


l_runtime = (agg_results.query("Size == 'L'")["avg_runtime"] * 15).sum()
l_all_runtime = s_m_runtime + l_runtime
l_all_cost = (s_m_runtime * c4_std_2_cost + l_runtime * c4_hgm_16_cost) / 3600
print(
    f"If we ran Ls on all solvers, total runtime = {l_all_runtime:,.0f}s ({naturaldelta(l_all_runtime, months=False)}), cost = ${l_all_cost:,.0f}"
)

Run v2, total runtime = 23,196,538s (268 days), cost = $5,956
If we ran Ls on all solvers, total runtime = 57,302,728s (1 year, 298 days), cost = $15,903


In [43]:
f"If we ran Ls on all solvers, total cost including discarded runs = ${(6300 / total_cost) * l_all_cost:,.0f}"

'If we ran Ls on all solvers, total cost including discarded runs = $16,823'